### Windowing operations
- From https://pandas.pydata.org/docs/user_guide/window.html
- pandas contains a compact set of APIs for performing windowing operations - an operation that performs an aggregation over a sliding partition of values. The API functions similarly to the groupby API in that Series and DataFrame call the windowing method with necessary parameters and then subsequently call the aggregation function.

In [2]:
import pandas as pd
import numpy as np

In [3]:
s = pd.Series(range(5))
s.rolling(window=2).sum()

0    NaN
1    1.0
2    3.0
3    5.0
4    7.0
dtype: float64

- The windows are comprised by looking back the length of the window from the current observation. The result above can be derived by taking the sum of the following windowed partitions of data:

In [4]:
for window in s.rolling(window=2):
    print(window)

0    0
dtype: int64
0    0
1    1
dtype: int64
1    1
2    2
dtype: int64
2    2
3    3
dtype: int64
3    3
4    4
dtype: int64


#### Overview
- pandas supports 4 types of windowing operations:
    - Rolling window: Generic fixed or variable sliding window over the values.
    - Weighted window: Weighted, non-rectangular window supplied by the scipy.signal library.
    - Expanding window: Accumulating window over the values.
    - Exponentially Weighted window: Accumulating and exponentially weighted window over the values.

|  Concept |  Method | Returned Object | Supports time-based windows | Supports chained groupby | Supports table method | Supports online operations |
| --- | --- | --- | --- | --- | --- | --- |
| Rolling window | rolling | pandas.typing.api.Rolling | Yes | Yes | Yes (as of version 1.3) |  No |
| Weighted window | rolling | pandas.typing.api.Window | No | No | No | No |
| Expanding window | expanding | pandas.typing.api.Expanding | No | Yes | Yes (as of version 1.3) | No |
| Exponentially Weighted window | ewm | pandas.typing.api.ExponentialMovingWindow |No | Yes (as of version 1.2) | No  | Yes (as of version 1.3) |

- As noted above, some operations support specifying a window based on a time offset:

In [6]:
s = pd.Series(range(5), index=pd.date_range('2020-01-01', periods=5, freq='1D'))

s.rolling(window="2d").sum()

2020-01-01    0.0
2020-01-02    1.0
2020-01-03    3.0
2020-01-04    5.0
2020-01-05    7.0
Freq: D, dtype: float64

- Additionally, some methods support chaining a groupby operation with a windowing operation which will first group the data by the specified keys and then perform a windowing operation per group.

In [7]:
df = pd.DataFrame({'A': ['a', 'b', 'a', 'b', 'a'], 'B': range(5)})
df.groupby("A").expanding().sum()

B
A       
a 0  0.0
  2  2.0
  4  6.0
b 1  1.0
  3  4.0

- **Note**: Windowing operations currently only support numeric data (integer and float) and will always return float64 values.

- **Warning**: Some windowing aggregation, mean, sum, var and std methods may suffer from numerical imprecision due to the underlying windowing algorithms accumulating sums. When values differ with magnitude 
 this results in truncation. It must be noted, that large values may have an impact on windows, which do not include these values. Kahan summation is used to compute the rolling sums to preserve accuracy as much as possible.

- Some windowing operations also support the method='table' option in the constructor which performs the windowing operation over an entire DataFrame instead of a single column or row at a time. This can provide a useful performance benefit for a DataFrame with many columns or rows (with the corresponding axis argument) or the ability to utilize other columns during the windowing operation. The method='table' option can only be used if engine='numba' is specified in the corresponding method call.

- For example, a weighted mean calculation can be calculated with apply() by specifying a separate column of weights.

In [8]:
def weighted_mean(x):
    arr = np.ones((1, x.shape[1]))
    arr[:, :2] = (x[:, :2] * x[:, 2]).sum(axis=0) / x[:, 2].sum()
    return arr

df = pd.DataFrame([[1, 2, 0.6], [2, 3, 0.4], [3, 4, 0.2], [4, 5, 0.7]])

df.rolling(2, method="table", min_periods=0).apply(weighted_mean, raw=True, engine="numba")  # noqa: E501


,0,1,2
0,1.000000,2.000000,1.0
1,1.800000,2.000000,1.0
2,3.333333,2.333333,1.0
3,1.555556,7.000000,1.0


- Some windowing operations also support an online method after constructing a windowing object which returns a new object that supports passing in new DataFrame or Series objects to continue the windowing calculation with the new values (i.e. online calculations).

- The methods on this new windowing objects must call the aggregation method first to “prime” the initial state of the online calculation. Then, new DataFrame or Series objects can be passed in the update argument to continue the windowing calculation.

In [10]:
df = pd.DataFrame([[1, 2, 0.6], [2, 3, 0.4], [3, 4, 0.2], [4, 5, 0.7]])

df.ewm(0.5).mean()

,0,1,2
0,1.000000,2.000000,0.600000
1,1.750000,2.750000,0.450000
2,2.615385,3.615385,0.276923
3,3.550000,4.550000,0.562500


In [11]:
online_ewm = df.head(2).ewm(0.5).online()
online_ewm.mean()

,0,1,2
0,1.00,2.00,0.60
1,1.75,2.75,0.45


In [12]:
online_ewm.mean(update=df.tail(1))

,0,1,2
3,3.307692,4.307692,0.623077


- All windowing operations support a min_periods argument that dictates the minimum amount of non-np.nan values a window must have; otherwise, the resulting value is np.nan. min_periods defaults to 1 for time-based windows and window for fixed windows

In [13]:
s = pd.Series([np.nan, 1, 2, np.nan, np.nan, 3])
s.rolling(window=3, min_periods=1).sum()

0    NaN
1    1.0
2    3.0
3    3.0
4    2.0
5    3.0
dtype: float64

In [14]:
s.rolling(window=3, min_periods=2).sum()

0    NaN
1    NaN
2    3.0
3    3.0
4    NaN
5    NaN
dtype: float64

In [15]:
# Equivalent to min_periods=3
s.rolling(window=3, min_periods=None).sum()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
dtype: float64

- Additionally, all windowing operations supports the aggregate method for returning a result of multiple aggregations applied to a window.

In [16]:
df = pd.DataFrame({"A": range(5), "B": range(10, 15)})
df.expanding().agg(["sum", "mean", "std"])

A                    B                
    sum mean       std   sum  mean       std
0   0.0  0.0       NaN  10.0  10.0       NaN
1   1.0  0.5  0.707107  21.0  10.5  0.707107
2   3.0  1.0  1.000000  33.0  11.0  1.000000
3   6.0  1.5  1.290994  46.0  11.5  1.290994
4  10.0  2.0  1.581139  60.0  12.0  1.581139

#### Rolling window
- Generic rolling windows support specifying windows as a fixed number of observations or variable number of observations based on an offset. If a time based offset is provided, the corresponding time based index must be monotonic.

In [17]:
times = ['2020-01-01', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-29']
s = pd.Series(range(5), index=pd.DatetimeIndex(times))
s

2020-01-01    0
2020-01-03    1
2020-01-04    2
2020-01-05    3
2020-01-29    4
dtype: int64

In [18]:
# Window with 2 observations
s.rolling(window=2).sum()

2020-01-01    NaN
2020-01-03    1.0
2020-01-04    3.0
2020-01-05    5.0
2020-01-29    7.0
dtype: float64

In [19]:
# Window with 2 days worth of observations
s.rolling(window='2D').sum()

2020-01-01    0.0
2020-01-03    1.0
2020-01-04    3.0
2020-01-05    5.0
2020-01-29    4.0
dtype: float64

#### Centering windows
- By default the labels are set to the right edge of the window, but a center keyword is available so the labels can be set at the center.

In [20]:
s = pd.Series(range(10))
s.rolling(window=5).mean()

0    NaN
1    NaN
2    NaN
3    NaN
4    2.0
5    3.0
6    4.0
7    5.0
8    6.0
9    7.0
dtype: float64

In [21]:
s.rolling(window=5, center=True).mean()

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    NaN
9    NaN
dtype: float64

- This can also be applied to datetime-like indices.

In [22]:
df = pd.DataFrame(
    {"A": [0, 1, 2, 3, 4]}, index=pd.date_range("2020", periods=5, freq="1D")
)
df

,A
2020-01-01,0
2020-01-02,1
2020-01-03,2
2020-01-04,3
2020-01-05,4


In [23]:
df.rolling("2D", center=False).mean()

,A
2020-01-01,0.0
2020-01-02,0.5
2020-01-03,1.5
2020-01-04,2.5
2020-01-05,3.5


In [24]:
df.rolling("2D", center=True).mean()

,A
2020-01-01,0.5
2020-01-02,1.5
2020-01-03,2.5
2020-01-04,3.5
2020-01-05,4.0


#### Rolling window endpoints
- The inclusion of the interval endpoints in rolling window calculations can be specified with the closed parameter:

| Value | Behavior |
| --- | ---- |
| 'right' | close right endpoint |
| 'left' | close left endpoint |
| 'both' | close both endpoints |
| 'neither' | open endpoints |

- For example, having the right endpoint open is useful in many problems that require that there is no contamination from present information back to past information. This allows the rolling window to compute statistics “up to that point in time”, but not including that point in time.

In [25]:
df = pd.DataFrame(
    {"x": 1},
    index=[
        pd.Timestamp("20130101 09:00:01"),
        pd.Timestamp("20130101 09:00:02"),
        pd.Timestamp("20130101 09:00:03"),
        pd.Timestamp("20130101 09:00:04"),
        pd.Timestamp("20130101 09:00:06"),
    ],
)
df["right"] = df.rolling("2s", closed="right").x.sum()  # default
df["both"] = df.rolling("2s", closed="both").x.sum()
df["left"] = df.rolling("2s", closed="left").x.sum()
df["neither"] = df.rolling("2s", closed="neither").x.sum()
df

,x,right,both,left,neither
2013-01-01 09:00:01,1,1.0,1.0,NaN,NaN
2013-01-01 09:00:02,1,2.0,2.0,1.0,1.0
2013-01-01 09:00:03,1,2.0,3.0,2.0,1.0
2013-01-01 09:00:04,1,2.0,3.0,2.0,1.0
2013-01-01 09:00:06,1,1.0,2.0,1.0,NaN


#### Custom window rolling
- In addition to accepting an integer or offset as a window argument, rolling also accepts a BaseIndexer subclass that allows a user to define a custom method for calculating window bounds. The BaseIndexer subclass will need to define a get_window_bounds method that returns a tuple of two arrays, the first being the starting indices of the windows and second being the ending indices of the windows. Additionally, num_values, min_periods, center, closed and step will automatically be passed to get_window_bounds and the defined method must always accept these arguments.

- For example, if we have the following DataFrame

In [26]:
use_expanding = [True, False, True, False, True]
use_expanding

[True, False, True, False, True]

In [27]:
df = pd.DataFrame({"values": range(5)})
df

,values
0,0
1,1
2,2
3,3
4,4


- and we want to use an expanding window where use_expanding is True otherwise a window of size 1, we can create the following BaseIndexer subclass:

In [28]:
from pandas.api.indexers import BaseIndexer

class CustomIndexer(BaseIndexer):
     def get_window_bounds(self, num_values, min_periods, center, closed, step):
         start = np.empty(num_values, dtype=np.int64)
         end = np.empty(num_values, dtype=np.int64)
         for i in range(num_values):
             if self.use_expanding[i]:
                 start[i] = 0
                 end[i] = i + 1
             else:
                 start[i] = i
                 end[i] = i + self.window_size
         return start, end

indexer = CustomIndexer(window_size=1, use_expanding=use_expanding)
df.rolling(indexer).sum()


,values
0,0.0
1,1.0
2,3.0
3,3.0
4,10.0


- You can view other examples of BaseIndexer subclasses here

- One subclass of note within those examples is the VariableOffsetWindowIndexer that allows rolling operations over a non-fixed offset like a BusinessDay.

In [29]:
from pandas.api.indexers import VariableOffsetWindowIndexer
df = pd.DataFrame(range(10), index=pd.date_range("2020", periods=10))

offset = pd.offsets.BDay(1)
indexer = VariableOffsetWindowIndexer(index=df.index, offset=offset)
df

,0
2020-01-01,0
2020-01-02,1
2020-01-03,2
2020-01-04,3
2020-01-05,4
2020-01-06,5
2020-01-07,6
2020-01-08,7
2020-01-09,8
2020-01-10,9


In [30]:
df.rolling(indexer).sum()

,0
2020-01-01,0.0
2020-01-02,1.0
2020-01-03,2.0
2020-01-04,3.0
2020-01-05,7.0
2020-01-06,12.0
2020-01-07,6.0
2020-01-08,7.0
2020-01-09,8.0
2020-01-10,9.0


- For some problems knowledge of the future is available for analysis. For example, this occurs when each data point is a full time series read from an experiment, and the task is to extract underlying conditions. In these cases it can be useful to perform forward-looking rolling window computations. FixedForwardWindowIndexer class is available for this purpose. This BaseIndexer subclass implements a closed fixed-width forward-looking rolling window, and we can use it as follows:

In [31]:
from pandas.api.indexers import FixedForwardWindowIndexer
indexer = FixedForwardWindowIndexer(window_size=2)
df.rolling(indexer, min_periods=1).sum()

,0
2020-01-01,1.0
2020-01-02,3.0
2020-01-03,5.0
2020-01-04,7.0
2020-01-05,9.0
2020-01-06,11.0
2020-01-07,13.0
2020-01-08,15.0
2020-01-09,17.0
2020-01-10,9.0


- We can also achieve this by using slicing, applying rolling aggregation, and then flipping the result as shown in example below:

In [32]:
df = pd.DataFrame(
    data=[
        [pd.Timestamp("2018-01-01 00:00:00"), 100],
        [pd.Timestamp("2018-01-01 00:00:01"), 101],
        [pd.Timestamp("2018-01-01 00:00:03"), 103],
        [pd.Timestamp("2018-01-01 00:00:04"), 111],
    ],
    columns=["time", "value"],
).set_index("time")
df

,value
time,
2018-01-01 00:00:00,100
2018-01-01 00:00:01,101
2018-01-01 00:00:03,103
2018-01-01 00:00:04,111


In [33]:
reversed_df = df[::-1].rolling("2s").sum()[::-1]
reversed_df

,value
time,
2018-01-01 00:00:00,201.0
2018-01-01 00:00:01,101.0
2018-01-01 00:00:03,214.0
2018-01-01 00:00:04,111.0


#### Rolling apply
- The apply() function takes an extra func argument and performs generic rolling computations. The func argument should be a single function that produces a single value from an ndarray input. raw specifies whether the windows are cast as Series objects (raw=False) or ndarray objects (raw=True).

In [34]:
def mad(x):
    return np.fabs(x - x.mean()).mean()

s = pd.Series(range(10))
s.rolling(window=4).apply(mad, raw=True)

0    NaN
1    NaN
2    NaN
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
dtype: float64

#### Numba engine
- Additionally, apply() can leverage Numba if installed as an optional dependency. The apply aggregation can be executed using Numba by specifying engine='numba' and engine_kwargs arguments (raw must also be set to True). See enhancing performance with Numba for general usage of the arguments and performance considerations.

- Numba will be applied in potentially two routines:
    - If func is a standard Python function, the engine will JIT the passed function. func can also be a JITed function in which case the engine will not JIT the function again.
    - The engine will JIT the for loop where the apply function is applied to each window.

- The engine_kwargs argument is a dictionary of keyword arguments that will be passed into the numba.jit decorator. These keyword arguments will be applied to both the passed function (if a standard Python function) and the apply for loop over each window.

- mean, median, max, min, and sum also support the engine and engine_kwargs arguments.

#### Binary window functions
- cov() and corr() can compute moving window statistics about two Series or any combination of DataFrame/Series or DataFrame/DataFrame. Here is the behavior in each case:
    - two Series: compute the statistic for the pairing.
    - DataFrame/Series: compute the statistics for each column of the DataFrame with the passed Series, thus returning a DataFrame.
    - DataFrame/DataFrame: by default compute the statistic for matching column names, returning a DataFrame. If the keyword argument pairwise=True is passed then computes the statistic for each pair of columns, returning a DataFrame with a MultiIndex whose values are the dates in question (see the next section).

- For example:

In [35]:
df = pd.DataFrame(
    np.random.randn(10, 4),
    index=pd.date_range("2020-01-01", periods=10),
    columns=["A", "B", "C", "D"],
)
df = df.cumsum()
df2 = df[:4]
df2.rolling(window=2).corr(df2["B"])

,A,B,C,D
2020-01-01,NaN,NaN,NaN,NaN
2020-01-02,1.0,1.0,1.0,-1.0
2020-01-03,1.0,1.0,1.0,-1.0
2020-01-04,-1.0,1.0,-1.0,-1.0


#### Computing rolling pairwise covariances and correlations
- In financial data analysis and other fields it’s common to compute covariance and correlation matrices for a collection of time series. Often one is also interested in moving-window covariance and correlation matrices. This can be done by passing the pairwise keyword argument, which in the case of DataFrame inputs will yield a MultiIndexed DataFrame whose index are the dates in question. In the case of a single DataFrame argument the pairwise argument can even be omitted:

- **Note**:
- Missing values are ignored and each entry is computed using the pairwise complete observations.

- Assuming the missing data are missing at random this results in an estimate for the covariance matrix which is unbiased. However, for many applications this estimate may not be acceptable because the estimated covariance matrix is not guaranteed to be positive semi-definite. This could lead to estimated correlations having absolute values which are greater than one, and/or a non-invertible covariance matrix. See Estimation of covariance matrices for more details.

In [36]:
covs = (
    df[["B", "C", "D"]]
    .rolling(window=4)
    .cov(df[["A", "B", "C"]], pairwise=True)
)

covs

B         C         D
2020-01-01 A       NaN       NaN       NaN
           B       NaN       NaN       NaN
           C       NaN       NaN       NaN
2020-01-02 A       NaN       NaN       NaN
           B       NaN       NaN       NaN
           C       NaN       NaN       NaN
2020-01-03 A       NaN       NaN       NaN
           B       NaN       NaN       NaN
           C       NaN       NaN       NaN
2020-01-04 A  0.008265  0.844142 -0.246741
           B  0.227966  0.121286 -0.268651
           C  0.121286  2.110627 -0.190263
2020-01-05 A -0.240157  1.564655 -0.717099
           B  0.671121 -0.855186  0.453196
           C -0.855186  3.008593 -1.305012
2020-01-06 A -1.065921  1.159806 -0.579214
           B  1.771164 -1.856435  1.204398
           C -1.856435  2.198318 -0.944005
2020-01-07 A -2.003297  0.064349 -1.197044
           B  3.277875 -0.083180  2.199326
           C -0.083180  0.438349 -0.343330
2020-01-08 A -2.874049 -1.037109  0.748213
           B  2.791650  0.983037 -0.259602
           C  0.983037  0.640702 -0.308023
2020-01-09 A -1.608907 -1.386361  1.268514
           B  1.511788  1.219597 -0.665467
           C  1.219597  1.023215 -0.672874
2020-01-10 A -0.222339 -0.315117  0.860012
           B  0.157861  0.213175 -0.210837
           C  0.213175  0.294630 -0.307451

#### Weighted window
- The win_type argument in .rolling generates a weighted windows that are commonly used in filtering and spectral estimation. win_type must be string that corresponds to a scipy.signal window function. Scipy must be installed in order to use these windows, and supplementary arguments that the Scipy window methods take must be specified in the aggregation function.



In [37]:
s = pd.Series(range(10))
s.rolling(window=5).mean()

0    NaN
1    NaN
2    NaN
3    NaN
4    2.0
5    3.0
6    4.0
7    5.0
8    6.0
9    7.0
dtype: float64

In [38]:
s.rolling(window=5, win_type="triang").mean()

0    NaN
1    NaN
2    NaN
3    NaN
4    2.0
5    3.0
6    4.0
7    5.0
8    6.0
9    7.0
dtype: float64

In [39]:
# Supplementary Scipy arguments passed in the aggregation function
s.rolling(window=5, win_type="gaussian").mean(std=0.1)

0    NaN
1    NaN
2    NaN
3    NaN
4    2.0
5    3.0
6    4.0
7    5.0
8    6.0
9    7.0
dtype: float64

#### Expanding window
- An expanding window yields the value of an aggregation statistic with all the data available up to that point in time. Since these calculations are a special case of rolling statistics, they are implemented in pandas such that the following two calls are equivalent:

In [40]:
df = pd.DataFrame(range(5))
df.rolling(window=len(df), min_periods=1).mean()

,0
0,0.0
1,0.5
2,1.0
3,1.5
4,2.0


In [41]:
df.expanding(min_periods=1).mean()

,0
0,0.0
1,0.5
2,1.0
3,1.5
4,2.0


- Skip: Exponentially weighted window

#### Time series / date functionality
- From https://pandas.pydata.org/docs/user_guide/timeseries.html
- pandas contains extensive capabilities and features for working with time series data for all domains. Using the NumPy datetime64 and timedelta64 dtypes, pandas has consolidated a large number of features from other Python libraries like scikits.timeseries as well as created a tremendous amount of new functionality for manipulating time series data.

- For example, pandas supports:

- Parsing time series information from various sources and formats

In [42]:
import datetime

dti = pd.to_datetime(
    ["1/1/2018", np.datetime64("2018-01-01"), datetime.datetime(2018, 1, 1)]
)

dti

DatetimeIndex(['2018-01-01', '2018-01-01', '2018-01-01'], dtype='datetime64[ns]', freq=None)

- Generate sequences of fixed-frequency dates and time spans

In [43]:
dti = pd.date_range("2018-01-01", periods=3, freq="h")
dti

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00'],
              dtype='datetime64[ns]', freq='h')

- Manipulating and converting date times with timezone information

In [44]:
dti = dti.tz_localize("UTC")
dti

DatetimeIndex(['2018-01-01 00:00:00+00:00', '2018-01-01 01:00:00+00:00',
               '2018-01-01 02:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='h')

In [45]:
dti.tz_convert("US/Pacific")

DatetimeIndex(['2017-12-31 16:00:00-08:00', '2017-12-31 17:00:00-08:00',
               '2017-12-31 18:00:00-08:00'],
              dtype='datetime64[ns, US/Pacific]', freq='h')

- Resampling or converting a time series to a particular frequency

In [46]:
idx = pd.date_range("2018-01-01", periods=5, freq="h")
ts = pd.Series(range(len(idx)), index=idx)
ts

2018-01-01 00:00:00    0
2018-01-01 01:00:00    1
2018-01-01 02:00:00    2
2018-01-01 03:00:00    3
2018-01-01 04:00:00    4
Freq: h, dtype: int64

In [47]:
ts.resample("2h").mean()

2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2h, dtype: float64

- Performing date and time arithmetic with absolute or relative time increments

In [48]:
friday = pd.Timestamp("2018-01-05")
friday.day_name()

'Friday'

In [49]:
# Add 1 day
saturday = friday + pd.Timedelta("1 day")
saturday.day_name()

'Saturday'

In [50]:
# Add 1 business day (Friday --> Monday)
monday = friday + pd.offsets.BDay()
monday.day_name()

'Monday'

- pandas provides a relatively compact and self-contained set of tools for performing the above tasks and more.

#### Overview
- pandas captures 4 general time related concepts:
    - Date times: A specific date and time with timezone support. Similar to datetime.datetime from the standard library.
    - Time deltas: An absolute time duration. Similar to datetime.timedelta from the standard library.
    - Time spans: A span of time defined by a point in time and its associated frequency.
    - Date offsets: A relative time duration that respects calendar arithmetic. Similar to dateutil.relativedelta.relativedelta from the dateutil package.

| Concept |  Scalar Class | Array Class | pandas Data Type | Primary Creation Method |
| --- | --- | --- | ---- | ---- |
| Date times | Timestamp | DatetimeIndex | datetime64[ns] or datetime64[ns, tz] |to_datetime or date_range |
| Time deltas | Timedelta | TimedeltaIndex | timedelta64[ns] | to_timedelta or timedelta_range |
| Time spans | Period | PeriodIndex | period[freq] | Period or period_range |
| Date offsets | DateOffset | None | None | DateOffset|

- For time series data, it’s conventional to represent the time component in the index of a Series or DataFrame so manipulations can be performed with respect to the time element.

In [51]:
pd.Series(range(3), index=pd.date_range("2000", freq="D", periods=3))

2000-01-01    0
2000-01-02    1
2000-01-03    2
Freq: D, dtype: int64

- However, Series and DataFrame can directly also support the time component as data itself.

In [52]:
pd.Series(pd.date_range("2000", freq="D", periods=3))

0   2000-01-01
1   2000-01-02
2   2000-01-03
dtype: datetime64[ns]

- Series and DataFrame have extended data type support and functionality for datetime, timedelta and Period data when passed into those constructors. DateOffset data however will be stored as object data.

In [53]:
pd.Series(pd.period_range("1/1/2011", freq="M", periods=3))

0    2011-01
1    2011-02
2    2011-03
dtype: period[M]

In [54]:
pd.Series([pd.DateOffset(1), pd.DateOffset(2)])

0         <DateOffset>
1    <2 * DateOffsets>
dtype: object

In [55]:
pd.Series(pd.date_range("1/1/2011", freq="ME", periods=3))

0   2011-01-31
1   2011-02-28
2   2011-03-31
dtype: datetime64[ns]

- Lastly, pandas represents null date times, time deltas, and time spans as NaT which is useful for representing missing or null date like values and behaves similar as np.nan does for float data.

In [56]:
pd.Timestamp(pd.NaT)

NaT

In [57]:
pd.Timedelta(pd.NaT)

NaT

In [58]:
pd.Period(pd.NaT)

NaT

In [59]:
# Equality acts as np.nan would
pd.NaT == pd.NaT

False

#### Timestamps vs. time spans
- Timestamped data is the most basic type of time series data that associates values with points in time. For pandas objects it means using the points in time.

In [60]:
import datetime

pd.Timestamp(datetime.datetime(2012, 5, 1))



Timestamp('2012-05-01 00:00:00')

In [61]:
pd.Timestamp("2012-05-01")

Timestamp('2012-05-01 00:00:00')

In [62]:
pd.Timestamp(2012, 5, 1)

Timestamp('2012-05-01 00:00:00')

- However, in many cases it is more natural to associate things like change variables with a time span instead. The span represented by Period can be specified explicitly, or inferred from datetime string format.

- For example:

In [63]:
pd.Period("2011-01")

Period('2011-01', 'M')

In [64]:
pd.Period("2012-05", freq="D")

Period('2012-05-01', 'D')

- Timestamp and Period can serve as an index. Lists of Timestamp and Period are automatically coerced to DatetimeIndex and PeriodIndex respectively.

In [65]:
dates = [
    pd.Timestamp("2012-05-01"),
    pd.Timestamp("2012-05-02"),
    pd.Timestamp("2012-05-03"),
]

ts = pd.Series(np.random.randn(3), dates)
type(ts.index)


pandas.core.indexes.datetimes.DatetimeIndex

In [66]:
ts.index


DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

In [67]:
ts

2012-05-01   -1.374929
2012-05-02    0.714788
2012-05-03    0.503807
dtype: float64

In [68]:
periods = [pd.Period("2012-01"), pd.Period("2012-02"), pd.Period("2012-03")]

ts = pd.Series(np.random.randn(3), periods)

type(ts.index)

pandas.core.indexes.period.PeriodIndex

In [69]:
ts.index

PeriodIndex(['2012-01', '2012-02', '2012-03'], dtype='period[M]')

In [70]:
ts

2012-01   -1.717055
2012-02   -1.561730
2012-03    0.307990
Freq: M, dtype: float64

- pandas allows you to capture both representations and convert between them. Under the hood, pandas represents timestamps using instances of Timestamp and sequences of timestamps using instances of DatetimeIndex. For regular time spans, pandas uses Period objects for scalar values and PeriodIndex for sequences of spans. Better support for irregular intervals with arbitrary start and end points are forth-coming in future releases.

#### Converting to timestamps
- To convert a Series or list-like object of date-like objects e.g. strings, epochs, or a mixture, you can use the to_datetime function. When passed a Series, this returns a Series (with the same index), while a list-like is converted to a DatetimeIndex:

In [71]:
pd.to_datetime(pd.Series(["Jul 31, 2009", "Jan 10, 2010", None]))

0   2009-07-31
1   2010-01-10
2          NaT
dtype: datetime64[ns]

In [72]:
pd.to_datetime(["2005/11/23", "2010/12/31"])

DatetimeIndex(['2005-11-23', '2010-12-31'], dtype='datetime64[ns]', freq=None)

- If you use dates which start with the day first (i.e. European style), you can pass the dayfirst flag:

In [73]:
pd.to_datetime(["04-01-2012 10:00"], dayfirst=True)

DatetimeIndex(['2012-01-04 10:00:00'], dtype='datetime64[ns]', freq=None)

In [74]:
pd.to_datetime(["04-14-2012 10:00"], dayfirst=True)

C:\Users\thotc\AppData\Local\Temp\ipykernel_49048\2353669186.py:1: UserWarning: Parsing dates in %m-%d-%Y %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  pd.to_datetime(["04-14-2012 10:00"], dayfirst=True)


DatetimeIndex(['2012-04-14 10:00:00'], dtype='datetime64[ns]', freq=None)

- **Warning**: You see in the above example that dayfirst isn’t strict. If a date can’t be parsed with the day being first it will be parsed as if dayfirst were False and a warning will also be raised.

- If you pass a single string to to_datetime, it returns a single Timestamp. Timestamp can also accept string input, but it doesn’t accept string parsing options like dayfirst or format, so use to_datetime if these are required.

In [75]:
pd.to_datetime("2010/11/12")

Timestamp('2010-11-12 00:00:00')

In [76]:
pd.Timestamp("2010/11/12")

Timestamp('2010-11-12 00:00:00')

- You can also use the DatetimeIndex constructor directly:

In [77]:
pd.DatetimeIndex(["2018-01-01", "2018-01-03", "2018-01-05"])

DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-05'], dtype='datetime64[ns]', freq=None)

- The string ‘infer’ can be passed in order to set the frequency of the index as the inferred frequency upon creation:

In [78]:
pd.DatetimeIndex(["2018-01-01", "2018-01-03", "2018-01-05"], freq="infer")


DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-05'], dtype='datetime64[ns]', freq='2D')

#### Providing a format argument
- In addition to the required datetime string, a format argument can be passed to ensure specific parsing. This could also potentially speed up the conversion considerably.

In [79]:
pd.to_datetime("2010/11/12", format="%Y/%m/%d")

Timestamp('2010-11-12 00:00:00')

In [80]:
pd.to_datetime("12-11-2010 00:00", format="%d-%m-%Y %H:%M")

Timestamp('2010-11-12 00:00:00')

#### Assembling datetime from multiple DataFrame columns
- You can also pass a DataFrame of integer or string columns to assemble into a Series of Timestamps.

In [81]:
df = pd.DataFrame(
    {"year": [2015, 2016], "month": [2, 3], "day": [4, 5], "hour": [2, 3]}
)
pd.to_datetime(df)

0   2015-02-04 02:00:00
1   2016-03-05 03:00:00
dtype: datetime64[ns]

- You can pass only the columns that you need to assemble.

In [82]:
pd.to_datetime(df[["year", "month", "day"]])

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]

- pd.to_datetime looks for standard designations of the datetime component in the column names, including:
    - required: year, month, day
    - optional: hour, minute, second, millisecond, microsecond, nanosecond

#### Invalid data
- The default behavior, errors='raise', is to raise when unparsable:

In [83]:
try:
    pd.to_datetime(['2009/07/31', 'asd'], errors='raise')
except ValueError as e:
    print(e)

time data "asd" doesn't match format "%Y/%m/%d", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.


- Pass errors='coerce' to convert unparsable data to NaT (not a time):

In [84]:
pd.to_datetime(["2009/07/31", "asd"], errors="coerce")

DatetimeIndex(['2009-07-31', 'NaT'], dtype='datetime64[ns]', freq=None)

#### Epoch timestamps
- pandas supports converting integer or float epoch times to Timestamp and DatetimeIndex. The default unit is nanoseconds, since that is how Timestamp objects are stored internally. However, epochs are often stored in another unit which can be specified. These are computed from the starting point specified by the origin parameter.

In [85]:
pd.to_datetime(
    [1349720105, 1349806505, 1349892905, 1349979305, 1350065705], unit="s"
)

DatetimeIndex(['2012-10-08 18:15:05', '2012-10-09 18:15:05',
               '2012-10-10 18:15:05', '2012-10-11 18:15:05',
               '2012-10-12 18:15:05'],
              dtype='datetime64[ns]', freq=None)

In [86]:
pd.to_datetime(
    [1349720105100, 1349720105200, 1349720105300, 1349720105400, 1349720105500],
    unit="ms",
)

DatetimeIndex(['2012-10-08 18:15:05.100000', '2012-10-08 18:15:05.200000',
               '2012-10-08 18:15:05.300000', '2012-10-08 18:15:05.400000',
               '2012-10-08 18:15:05.500000'],
              dtype='datetime64[ns]', freq=None)

- **Note**: The unit parameter does not use the same strings as the format parameter that was discussed above. The available units are listed on the documentation for pandas.to_datetime().

- Constructing a Timestamp or DatetimeIndex with an epoch timestamp with the tz argument specified will raise a ValueError. If you have epochs in wall time in another timezone, you can read the epochs as timezone-naive timestamps and then localize to the appropriate timezone:

In [87]:
pd.Timestamp(1262347200000000000).tz_localize("US/Pacific")

Timestamp('2010-01-01 12:00:00-0800', tz='US/Pacific')

In [88]:
pd.DatetimeIndex([1262347200000000000]).tz_localize("US/Pacific")

DatetimeIndex(['2010-01-01 12:00:00-08:00'], dtype='datetime64[ns, US/Pacific]', freq=None)

- **Note**: Epoch times will be rounded to the nearest nanosecond.
- **Warning**: Conversion of float epoch times can lead to inaccurate and unexpected results. Python floats have about 15 digits precision in decimal. Rounding during conversion from float to high precision Timestamp is unavoidable. The only way to achieve exact precision is to use a fixed-width types (e.g. an int64).

In [89]:
pd.to_datetime([1490195805.433, 1490195805.433502912], unit="s")


DatetimeIndex(['2017-03-22 15:16:45.433000088', '2017-03-22 15:16:45.433502913'], dtype='datetime64[ns]', freq=None)

In [90]:
pd.to_datetime(1490195805433502912, unit="ns")


Timestamp('2017-03-22 15:16:45.433502912')

#### From timestamps to epoch
- To invert the operation from above, namely, to convert from a Timestamp to a ‘unix’ epoch:

In [91]:
stamps = pd.date_range("2012-10-08 18:15:05", periods=4, freq="D")
stamps

DatetimeIndex(['2012-10-08 18:15:05', '2012-10-09 18:15:05',
               '2012-10-10 18:15:05', '2012-10-11 18:15:05'],
              dtype='datetime64[ns]', freq='D')

- We subtract the epoch (midnight at January 1, 1970 UTC) and then floor divide by the “unit” (1 second).

In [92]:
(stamps - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")

Index([1349720105, 1349806505, 1349892905, 1349979305], dtype='int64')

#### Using the origin parameter
- Using the origin parameter, one can specify an alternative starting point for creation of a DatetimeIndex. For example, to use 1960-01-01 as the starting date:

In [93]:
pd.to_datetime([1, 2, 3], unit="D", origin=pd.Timestamp("1960-01-01"))

DatetimeIndex(['1960-01-02', '1960-01-03', '1960-01-04'], dtype='datetime64[ns]', freq=None)

- The default is set at origin='unix', which defaults to 1970-01-01 00:00:00. Commonly called ‘unix epoch’ or POSIX time.

In [94]:
pd.to_datetime([1, 2, 3], unit="D")


DatetimeIndex(['1970-01-02', '1970-01-03', '1970-01-04'], dtype='datetime64[ns]', freq=None)

#### Generating ranges of timestamps
- To generate an index with timestamps, you can use either the DatetimeIndex or Index constructor and pass in a list of datetime objects:

In [95]:
dates = [
    datetime.datetime(2012, 5, 1),
    datetime.datetime(2012, 5, 2),
    datetime.datetime(2012, 5, 3),
]
# Note the frequency information
index = pd.DatetimeIndex(dates)
index


DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

In [96]:

# Automatically converted to DatetimeIndex
index = pd.Index(dates)
index

DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

- In practice this becomes very cumbersome because we often need a very long index with a large number of timestamps. If we need timestamps on a regular frequency, we can use the date_range() and bdate_range() functions to create a DatetimeIndex. The default frequency for date_range is a calendar day while the default for bdate_range is a business day:


In [97]:
start = datetime.datetime(2011, 1, 1)
end = datetime.datetime(2012, 1, 1)
index = pd.date_range(start, end)
index

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03', '2011-01-04',
               '2011-01-05', '2011-01-06', '2011-01-07', '2011-01-08',
               '2011-01-09', '2011-01-10',
               ...
               '2011-12-23', '2011-12-24', '2011-12-25', '2011-12-26',
               '2011-12-27', '2011-12-28', '2011-12-29', '2011-12-30',
               '2011-12-31', '2012-01-01'],
              dtype='datetime64[ns]', length=366, freq='D')

In [98]:
index = pd.bdate_range(start, end)
index

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14',
               ...
               '2011-12-19', '2011-12-20', '2011-12-21', '2011-12-22',
               '2011-12-23', '2011-12-26', '2011-12-27', '2011-12-28',
               '2011-12-29', '2011-12-30'],
              dtype='datetime64[ns]', length=260, freq='B')

- Convenience functions like date_range and bdate_range can utilize a variety of frequency aliases:

In [99]:
pd.date_range(start, periods=1000, freq="ME")

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-30',
               '2011-05-31', '2011-06-30', '2011-07-31', '2011-08-31',
               '2011-09-30', '2011-10-31',
               ...
               '2093-07-31', '2093-08-31', '2093-09-30', '2093-10-31',
               '2093-11-30', '2093-12-31', '2094-01-31', '2094-02-28',
               '2094-03-31', '2094-04-30'],
              dtype='datetime64[ns]', length=1000, freq='ME')

In [100]:
pd.bdate_range(start, periods=250, freq="BQS")

DatetimeIndex(['2011-01-03', '2011-04-01', '2011-07-01', '2011-10-03',
               '2012-01-02', '2012-04-02', '2012-07-02', '2012-10-01',
               '2013-01-01', '2013-04-01',
               ...
               '2071-01-01', '2071-04-01', '2071-07-01', '2071-10-01',
               '2072-01-01', '2072-04-01', '2072-07-01', '2072-10-03',
               '2073-01-02', '2073-04-03'],
              dtype='datetime64[ns]', length=250, freq='BQS-JAN')

- date_range and bdate_range make it easy to generate a range of dates using various combinations of parameters like start, end, periods, and freq. The start and end dates are strictly inclusive, so dates outside of those specified will not be generated:

In [101]:
pd.date_range(start, end, freq="BME")

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31', '2011-06-30', '2011-07-29', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-30'],
              dtype='datetime64[ns]', freq='BME')

In [102]:
pd.date_range(start, end, freq="W")

DatetimeIndex(['2011-01-02', '2011-01-09', '2011-01-16', '2011-01-23',
               '2011-01-30', '2011-02-06', '2011-02-13', '2011-02-20',
               '2011-02-27', '2011-03-06', '2011-03-13', '2011-03-20',
               '2011-03-27', '2011-04-03', '2011-04-10', '2011-04-17',
               '2011-04-24', '2011-05-01', '2011-05-08', '2011-05-15',
               '2011-05-22', '2011-05-29', '2011-06-05', '2011-06-12',
               '2011-06-19', '2011-06-26', '2011-07-03', '2011-07-10',
               '2011-07-17', '2011-07-24', '2011-07-31', '2011-08-07',
               '2011-08-14', '2011-08-21', '2011-08-28', '2011-09-04',
               '2011-09-11', '2011-09-18', '2011-09-25', '2011-10-02',
               '2011-10-09', '2011-10-16', '2011-10-23', '2011-10-30',
               '2011-11-06', '2011-11-13', '2011-11-20', '2011-11-27',
               '2011-12-04', '2011-12-11', '2011-12-18', '2011-12-25',
               '2012-01-01'],
              dtype='datetime64[ns]', freq='W-S

In [103]:
pd.bdate_range(end=end, periods=20)

DatetimeIndex(['2011-12-05', '2011-12-06', '2011-12-07', '2011-12-08',
               '2011-12-09', '2011-12-12', '2011-12-13', '2011-12-14',
               '2011-12-15', '2011-12-16', '2011-12-19', '2011-12-20',
               '2011-12-21', '2011-12-22', '2011-12-23', '2011-12-26',
               '2011-12-27', '2011-12-28', '2011-12-29', '2011-12-30'],
              dtype='datetime64[ns]', freq='B')

In [104]:
pd.bdate_range(start=start, periods=20)

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14', '2011-01-17', '2011-01-18',
               '2011-01-19', '2011-01-20', '2011-01-21', '2011-01-24',
               '2011-01-25', '2011-01-26', '2011-01-27', '2011-01-28'],
              dtype='datetime64[ns]', freq='B')

- Specifying start, end, and periods will generate a range of evenly spaced dates from start to end inclusively, with periods number of elements in the resulting DatetimeIndex:

In [105]:
pd.date_range("2018-01-01", "2018-01-05", periods=5)

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq=None)

In [106]:
pd.date_range("2018-01-01", "2018-01-05", periods=10)

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 10:40:00',
               '2018-01-01 21:20:00', '2018-01-02 08:00:00',
               '2018-01-02 18:40:00', '2018-01-03 05:20:00',
               '2018-01-03 16:00:00', '2018-01-04 02:40:00',
               '2018-01-04 13:20:00', '2018-01-05 00:00:00'],
              dtype='datetime64[ns]', freq=None)

#### Custom frequency ranges
- bdate_range can also generate a range of custom frequency dates by using the weekmask and holidays parameters. These parameters will only be used if a custom frequency string is passed.

In [107]:
weekmask = "Mon Wed Fri"
holidays = [datetime.datetime(2011, 1, 5), datetime.datetime(2011, 3, 14)]
pd.bdate_range(start, end, freq="C", weekmask=weekmask, holidays=holidays)



DatetimeIndex(['2011-01-03', '2011-01-07', '2011-01-10', '2011-01-12',
               '2011-01-14', '2011-01-17', '2011-01-19', '2011-01-21',
               '2011-01-24', '2011-01-26',
               ...
               '2011-12-09', '2011-12-12', '2011-12-14', '2011-12-16',
               '2011-12-19', '2011-12-21', '2011-12-23', '2011-12-26',
               '2011-12-28', '2011-12-30'],
              dtype='datetime64[ns]', length=154, freq='C')

In [108]:
pd.bdate_range(start, end, freq="CBMS", weekmask=weekmask)


DatetimeIndex(['2011-01-03', '2011-02-02', '2011-03-02', '2011-04-01',
               '2011-05-02', '2011-06-01', '2011-07-01', '2011-08-01',
               '2011-09-02', '2011-10-03', '2011-11-02', '2011-12-02'],
              dtype='datetime64[ns]', freq='CBMS')

#### Timestamp limitations
- The limits of timestamp representation depend on the chosen resolution. For nanosecond resolution, the time span that can be represented using a 64-bit integer is limited to approximately 584 years:

In [109]:
pd.Timestamp.min

Timestamp('1677-09-21 00:12:43.145224193')

In [110]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

- When choosing second-resolution, the available range grows to +/- 2.9e11 years. Different resolutions can be converted to each other through as_unit.

#### Indexing
- One of the main uses for DatetimeIndex is as an index for pandas objects. The DatetimeIndex class contains many time series related optimizations:
    - A large range of dates for various offsets are pre-computed and cached under the hood in order to make generating subsequent date ranges very fast (just have to grab a slice).
    - Fast shifting using the shift method on pandas objects.
    - Unioning of overlapping DatetimeIndex objects with the same frequency is very fast (important for fast data alignment).
    - Quick access to date fields via properties such as year, month, etc.
    - Regularization functions like snap and very fast asof logic.

- DatetimeIndex objects have all the basic functionality of regular Index objects, and a smorgasbord of advanced time series specific methods for easy frequency processing.

- **Note**: While pandas does not force you to have a sorted date index, some of these methods may have unexpected or incorrect behavior if the dates are unsorted.
- DatetimeIndex can be used like a regular index and offers all of its intelligent functionality like selection, slicing, etc.

In [111]:
rng = pd.date_range(start, end, freq="BME")
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts.index

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31', '2011-06-30', '2011-07-29', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-30'],
              dtype='datetime64[ns]', freq='BME')

In [112]:
ts[:5].index

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31'],
              dtype='datetime64[ns]', freq='BME')

In [113]:
ts[::2].index

DatetimeIndex(['2011-01-31', '2011-03-31', '2011-05-31', '2011-07-29',
               '2011-09-30', '2011-11-30'],
              dtype='datetime64[ns]', freq='2BME')

#### Partial string indexing
- Dates and strings that parse to timestamps can be passed as indexing parameters:

In [114]:
ts["1/31/2011"]

np.float64(-0.23876248159358382)

In [115]:
ts[datetime.datetime(2011, 12, 25):]

2011-12-30   -1.161605
Freq: BME, dtype: float64

In [116]:
ts["10/31/2011":"12/31/2011"]

2011-10-31   -0.226832
2011-11-30    1.827478
2011-12-30   -1.161605
Freq: BME, dtype: float64

- To provide convenience for accessing longer time series, you can also pass in the year or year and month as strings:

In [117]:
ts["2011"]

2011-01-31   -0.238762
2011-02-28   -0.261816
2011-03-31    0.100672
2011-04-29    0.354914
2011-05-31    1.787761
2011-06-30    1.762841
2011-07-29   -1.273535
2011-08-31    1.840718
2011-09-30    2.608156
2011-10-31   -0.226832
2011-11-30    1.827478
2011-12-30   -1.161605
Freq: BME, dtype: float64

In [118]:
ts["2011-6"]

2011-06-30    1.762841
Freq: BME, dtype: float64

- This type of slicing will work on a DataFrame with a DatetimeIndex as well. Since the partial string selection is a form of label slicing, the endpoints will be included. This would include matching times on an included date:
- **Warning**: Indexing DataFrame rows with a single string with getitem (e.g. frame[dtstring]) is deprecated starting with pandas 1.2.0 (given the ambiguity whether it is indexing the rows or selecting a column) and will be removed in a future version. The equivalent with .loc (e.g. frame.loc[dtstring]) is still supported.

In [119]:
dft = pd.DataFrame(
    np.random.randn(100000, 1),
    columns=["A"],
    index=pd.date_range("20130101", periods=100000, freq="min"),
)
dft

,A
2013-01-01 00:00:00,0.396624
2013-01-01 00:01:00,-1.259759
2013-01-01 00:02:00,-0.512173
2013-01-01 00:03:00,-0.189025
2013-01-01 00:04:00,3.341915
...,...
2013-03-11 10:35:00,-1.484396
2013-03-11 10:36:00,1.126302
2013-03-11 10:37:00,-0.295338
2013-03-11 10:38:00,0.152058


In [120]:
dft.loc["2013"]

,A
2013-01-01 00:00:00,0.396624
2013-01-01 00:01:00,-1.259759
2013-01-01 00:02:00,-0.512173
2013-01-01 00:03:00,-0.189025
2013-01-01 00:04:00,3.341915
...,...
2013-03-11 10:35:00,-1.484396
2013-03-11 10:36:00,1.126302
2013-03-11 10:37:00,-0.295338
2013-03-11 10:38:00,0.152058


- This starts on the very first time in the month, and includes the last date and time for the month:

In [121]:
dft["2013-1":"2013-2"]

,A
2013-01-01 00:00:00,0.396624
2013-01-01 00:01:00,-1.259759
2013-01-01 00:02:00,-0.512173
2013-01-01 00:03:00,-0.189025
2013-01-01 00:04:00,3.341915
...,...
2013-02-28 23:55:00,0.683056
2013-02-28 23:56:00,-1.330087
2013-02-28 23:57:00,-0.461849
2013-02-28 23:58:00,1.065314


- This specifies a stop time that includes all of the times on the last day:

In [122]:
dft["2013-1":"2013-2-28"]

,A
2013-01-01 00:00:00,0.396624
2013-01-01 00:01:00,-1.259759
2013-01-01 00:02:00,-0.512173
2013-01-01 00:03:00,-0.189025
2013-01-01 00:04:00,3.341915
...,...
2013-02-28 23:55:00,0.683056
2013-02-28 23:56:00,-1.330087
2013-02-28 23:57:00,-0.461849
2013-02-28 23:58:00,1.065314


- This specifies an exact stop time (and is not the same as the above):

In [123]:
dft["2013-1":"2013-2-28 00:00:00"]

,A
2013-01-01 00:00:00,0.396624
2013-01-01 00:01:00,-1.259759
2013-01-01 00:02:00,-0.512173
2013-01-01 00:03:00,-0.189025
2013-01-01 00:04:00,3.341915
...,...
2013-02-27 23:56:00,-0.787321
2013-02-27 23:57:00,1.484370
2013-02-27 23:58:00,0.460217
2013-02-27 23:59:00,0.679188


- We are stopping on the included end-point as it is part of the index:

In [124]:
dft["2013-1-15":"2013-1-15 12:30:00"]

,A
2013-01-15 00:00:00,-1.005468
2013-01-15 00:01:00,0.187553
2013-01-15 00:02:00,-0.459082
2013-01-15 00:03:00,0.210014
2013-01-15 00:04:00,0.423664
...,...
2013-01-15 12:26:00,0.127786
2013-01-15 12:27:00,-1.407641
2013-01-15 12:28:00,-2.646477
2013-01-15 12:29:00,0.710905


- DatetimeIndex partial string indexing also works on a DataFrame with a MultiIndex:

In [126]:
dft2 = pd.DataFrame(
    np.random.randn(20, 1),
    columns=["A"],
    index=pd.MultiIndex.from_product(
        [pd.date_range("20130101", periods=10, freq="12h"), ["a", "b"]]
    ),
)

dft2

A
2013-01-01 00:00:00 a  1.413401
                    b  1.255994
2013-01-01 12:00:00 a -0.772604
                    b  0.223255
2013-01-02 00:00:00 a  0.600653
                    b -0.260394
2013-01-02 12:00:00 a  1.227651
                    b  0.128300
2013-01-03 00:00:00 a -0.372551
                    b  0.647517
2013-01-03 12:00:00 a -0.294958
                    b  0.282980
2013-01-04 00:00:00 a -0.204583
                    b  0.538820
2013-01-04 12:00:00 a -1.117346
                    b -0.107478
2013-01-05 00:00:00 a -0.519826
                    b -0.854881
2013-01-05 12:00:00 a -0.677986
                    b  3.273020

In [127]:
dft2.loc["2013-01-05"]

A
2013-01-05 00:00:00 a -0.519826
                    b -0.854881
2013-01-05 12:00:00 a -0.677986
                    b  3.273020

In [128]:
idx = pd.IndexSlice
dft2 = dft2.swaplevel(0, 1).sort_index()
dft2.loc[idx[:, "2013-01-05"], :]

A
a 2013-01-05 00:00:00 -0.519826
  2013-01-05 12:00:00 -0.677986
b 2013-01-05 00:00:00 -0.854881
  2013-01-05 12:00:00  3.273020

- Slicing with string indexing also honors UTC offset.

In [129]:
df = pd.DataFrame([0], index=pd.DatetimeIndex(["2019-01-01"], tz="US/Pacific"))
df

,0
2019-01-01 00:00:00-08:00,0


In [130]:
df["2019-01-01 12:00:00+04:00":"2019-01-01 13:00:00+04:00"]

,0
2019-01-01 00:00:00-08:00,0


#### Slice vs. exact match
- The same string used as an indexing parameter can be treated either as a slice or as an exact match depending on the resolution of the index. If the string is less accurate than the index, it will be treated as a slice, otherwise as an exact match.

- Consider a Series object with a minute resolution index:

In [131]:
series_minute = pd.Series(
    [1, 2, 3],
    pd.DatetimeIndex(
        ["2011-12-31 23:59:00", "2012-01-01 00:00:00", "2012-01-01 00:02:00"]
    ),
)
series_minute.index.resolution

'minute'

- A timestamp string less accurate than a minute gives a Series object.

In [132]:
series_minute["2011-12-31 23"]

2011-12-31 23:59:00    1
dtype: int64

- A timestamp string with minute resolution (or more accurate), gives a scalar instead, i.e. it is not casted to a slice.

In [133]:
series_minute["2011-12-31 23:59"]

np.int64(1)

In [134]:
series_minute["2011-12-31 23:59:00"]

np.int64(1)

- If index resolution is second, then the minute-accurate timestamp gives a Series.

In [135]:
series_second = pd.Series(
    [1, 2, 3],
    pd.DatetimeIndex(
        ["2011-12-31 23:59:59", "2012-01-01 00:00:00", "2012-01-01 00:00:01"]
    ),
)
series_second.index.resolution

'second'

In [136]:
series_second["2011-12-31 23:59"]

2011-12-31 23:59:59    1
dtype: int64

- If the timestamp string is treated as a slice, it can be used to index DataFrame with .loc[] as well.

In [137]:
dft_minute = pd.DataFrame(
    {"a": [1, 2, 3], "b": [4, 5, 6]}, index=series_minute.index
)
dft_minute.loc["2011-12-31 23"]

,a,b
2011-12-31 23:59:00,1,4


- **Warning**: However, if the string is treated as an exact match, the selection in DataFrame’s [] will be column-wise and not row-wise, see Indexing Basics. For example dft_minute['2011-12-31 23:59'] will raise KeyError as '2012-12-31 23:59' has the same resolution as the index and there is no column with such name:

- To always have unambiguous selection, whether the row is treated as a slice or a single selection, use .loc.

In [138]:
dft_minute.loc["2011-12-31 23:59"]

a    1
b    4
Name: 2011-12-31 23:59:00, dtype: int64

- **Note** also that DatetimeIndex resolution cannot be less precise than day.

In [139]:
series_monthly = pd.Series(
    [1, 2, 3], pd.DatetimeIndex(["2011-12", "2012-01", "2012-02"])
)
series_monthly.index.resolution


'day'

In [140]:
series_monthly["2011-12"]  # returns Series

2011-12-01    1
dtype: int64

#### Exact indexing
- As discussed in previous section, indexing a DatetimeIndex with a partial string depends on the “accuracy” of the period, in other words how specific the interval is in relation to the resolution of the index. In contrast, indexing with Timestamp or datetime objects is exact, because the objects have exact meaning. These also follow the semantics of including both endpoints.

- These Timestamp and datetime objects have exact hours, minutes, and seconds, even though they were not explicitly specified (they are 0).

In [141]:
dft[datetime.datetime(2013, 1, 1): datetime.datetime(2013, 2, 28)]

,A
2013-01-01 00:00:00,0.396624
2013-01-01 00:01:00,-1.259759
2013-01-01 00:02:00,-0.512173
2013-01-01 00:03:00,-0.189025
2013-01-01 00:04:00,3.341915
...,...
2013-02-27 23:56:00,-0.787321
2013-02-27 23:57:00,1.484370
2013-02-27 23:58:00,0.460217
2013-02-27 23:59:00,0.679188


- With no defaults.

In [142]:
dft[
    datetime.datetime(2013, 1, 1, 10, 12, 0): datetime.datetime(
        2013, 2, 28, 10, 12, 0
    )
]

,A
2013-01-01 10:12:00,0.798171
2013-01-01 10:13:00,-0.198203
2013-01-01 10:14:00,-0.294018
2013-01-01 10:15:00,-0.512489
2013-01-01 10:16:00,-0.870823
...,...
2013-02-28 10:08:00,-0.477992
2013-02-28 10:09:00,1.033900
2013-02-28 10:10:00,0.944742
2013-02-28 10:11:00,-1.110249


#### Truncating & fancy indexing
- A truncate() convenience function is provided that is similar to slicing. Note that truncate assumes a 0 value for any unspecified date component in a DatetimeIndex in contrast to slicing which returns any partially matching dates:

In [143]:
rng2 = pd.date_range("2011-01-01", "2012-01-01", freq="W")
ts2 = pd.Series(np.random.randn(len(rng2)), index=rng2)
ts2.truncate(before="2011-11", after="2011-12")

2011-11-06    0.510362
2011-11-13    0.177046
2011-11-20    1.071384
2011-11-27   -1.679727
Freq: W-SUN, dtype: float64

In [144]:
ts2["2011-11":"2011-12"]

2011-11-06    0.510362
2011-11-13    0.177046
2011-11-20    1.071384
2011-11-27   -1.679727
2011-12-04    0.114637
2011-12-11    0.327276
2011-12-18   -0.187411
2011-12-25    0.515678
Freq: W-SUN, dtype: float64

- Even complicated fancy indexing that breaks the DatetimeIndex frequency regularity will result in a DatetimeIndex, although frequency is lost:

In [145]:
ts2.iloc[[0, 2, 6]].index

DatetimeIndex(['2011-01-02', '2011-01-16', '2011-02-13'], dtype='datetime64[ns]', freq=None)

#### Time/date components
- There are several time/date properties that one can access from Timestamp or a collection of timestamps like a DatetimeIndex.

| Property | Description |
| -------- | ---------- |
| year | The year of the datetime |
| month | The month of the datetime |
| day | The days of the datetime |
| hour | The hour of the datetime |
| minute | The minutes of the datetime |
| second | The seconds of the datetime |
| microsecond | The microseconds of the datetime |
| nanosecond | The nanoseconds of the datetime |
| date | Returns datetime.date (does not contain timezone information) |
| time | Returns datetime.time (does not contain timezone information) |
| timetz | Returns datetime.time as local time with timezone information |
| dayofyear | The ordinal day of year |
| day_of_year | The ordinal day of year |
| weekofyear | The week ordinal of the year |
| week | The week ordinal of the year |
| dayofweek | The number of the day of the week with Monday=0, Sunday=6 |
| day_of_week | The number of the day of the week with Monday=0, Sunday=6 |
| weekday | The number of the day of the week with Monday=0, Sunday=6 |
| quarter | Quarter of the date: Jan-Mar = 1, Apr-Jun = 2, etc.|
| days_in_month | The number of days in the month of the datetime |
| is_month_start | Logical indicating if first day of month (defined by frequency) |
| is_month_end | Logical indicating if last day of month (defined by frequency) |
| is_quarter_start | Logical indicating if first day of quarter (defined by frequency) |
| is_quarter_end | Logical indicating if last day of quarter (defined by frequency) |
| is_year_start | Logical indicating if first day of year (defined by frequency) |
| is_year_end | Logical indicating if last day of year (defined by frequency) |
| is_leap_year | Logical indicating if the date belongs to a leap year |

- Furthermore, if you have a Series with datetimelike values, then you can access these properties via the .dt accessor, as detailed in the section on .dt accessors.

- You may obtain the year, week and day components of the ISO year from the ISO 8601 standard:

In [146]:
idx = pd.date_range(start="2019-12-29", freq="D", periods=4)
idx.isocalendar()

,year,week,day
2019-12-29,2019,52,7
2019-12-30,2020,1,1
2019-12-31,2020,1,2
2020-01-01,2020,1,3


In [147]:
idx.to_series().dt.isocalendar()

,year,week,day
2019-12-29,2019,52,7
2019-12-30,2020,1,1
2019-12-31,2020,1,2
2020-01-01,2020,1,3


#### DateOffset objects
- In the preceding examples, frequency strings (e.g. 'D') were used to specify a frequency that defined:
    - how the date times in DatetimeIndex were spaced when using date_range()
    - the frequency of a Period or PeriodIndex

- These frequency strings map to a DateOffset object and its subclasses. A DateOffset is similar to a Timedelta that represents a duration of time but follows specific calendar duration rules. For example, a Timedelta day will always increment datetimes by 24 hours, while a DateOffset day will increment datetimes to the same time the next day whether a day represents 23, 24 or 25 hours due to daylight savings time. However, all DateOffset subclasses that are an hour or smaller (Hour, Minute, Second, Milli, Micro, Nano) behave like Timedelta and respect absolute time.

- The basic DateOffset acts similar to dateutil.relativedelta (relativedelta documentation) that shifts a date time by the corresponding calendar duration specified. The arithmetic operator (+) can be used to perform the shift.

In [149]:
# This particular day contains a day light savings time transition
ts = pd.Timestamp("2016-10-30 00:00:00", tz="Europe/Helsinki")
# Respects absolute time
ts + pd.Timedelta(days=1)


Timestamp('2016-10-30 23:00:00+0200', tz='Europe/Helsinki')

In [150]:
# Respects calendar time
ts + pd.DateOffset(days=1)


Timestamp('2016-10-31 00:00:00+0200', tz='Europe/Helsinki')

In [151]:
friday = pd.Timestamp("2018-01-05")
friday.day_name()

'Friday'

In [152]:
# Add 2 business days (Friday --> Tuesday)
two_business_days = 2 * pd.offsets.BDay()
friday + two_business_days

Timestamp('2018-01-09 00:00:00')

In [153]:
(friday + two_business_days).day_name()

'Tuesday'

- Most DateOffsets have associated frequencies strings, or offset aliases, that can be passed into freq keyword arguments. The available date offsets and associated frequency strings can be found below:

| Date Offset | Frequency String | Description |
| ------ | ------ | ------ |
| DateOffset | None | Generic offset class, defaults to absolute 24 hours |
| BDay or BusinessDay | 'B' | business day (weekday) |
| CDay or CustomBusinessDay | 'C' | custom business day |
| Week | 'W' | one week, optionally anchored on a day of the week |
| WeekOfMonth | 'WOM' | the x-th day of the y-th week of each month |
| LastWeekOfMonth | 'LWOM' | the x-th day of the last week of each month |
| MonthEnd | 'ME' | calendar month end |
| MonthBegin | 'MS' |  calendar month begin |
| BMonthEnd or BusinessMonthEnd | 'BME' | business month end |
| BMonthBegin or BusinessMonthBegin | 'BMS' | business month begin |
| CBMonthEnd or CustomBusinessMonthEnd | 'CBME' | custom business month end |
| CBMonthBegin or CustomBusinessMonthBegin | 'CBMS' | custom business month begin |
| SemiMonthEnd | 'SME' | 15th (or other day_of_month) and calendar month end |
| SemiMonthBegin | 'SMS' | 15th (or other day_of_month) and calendar month begin |
| QuarterEnd | 'QE' | calendar quarter end |
| QuarterBegin | 'QS' | calendar quarter begin |
| BQuarterEnd | 'BQE | business quarter end |
| BQuarterBegin | 'BQS' | business quarter begin |
| FY5253Quarter | 'REQ' | retail (aka 52-53 week) quarter |
| YearEnd | 'YE' | calendar year end |
| YearBegin | 'YS' or 'BYS' | calendar year begin |
| BYearEnd | 'BYE' | business year end |
| BYearBegin | 'BYS' | business year begin |
| FY5253 | 'RE' | retail (aka 52-53 week) year |
| Easter | None | Easter holiday |
| BusinessHour | 'bh' | business hour |
| CustomBusinessHour | 'cbh' | custom business hour |
| Day | 'D' | one absolute day |
| Hour | 'h' | one hour |
| Minute | 'min' | one minute |
| Second | 's' | one second |
| Milli | 'ms' | one millisecond |
| Micro | 'us' | one microsecond |
| Nano | 'ns' | one nanosecond |

- DateOffsets additionally have rollforward() and rollback() methods for moving a date forward or backward respectively to a valid offset date relative to the offset. For example, business offsets will roll dates that land on the weekends (Saturday and Sunday) forward to Monday since business offsets operate on the weekdays.

In [154]:
ts = pd.Timestamp("2018-01-06 00:00:00")
ts.day_name()

'Saturday'

In [155]:
# BusinessHour's valid offset dates are Monday through Friday
offset = pd.offsets.BusinessHour(start="09:00")
# Bring the date to the closest offset date (Monday)
offset.rollforward(ts)

Timestamp('2018-01-08 09:00:00')

In [156]:
# Date is brought to the closest offset date first and then the hour is added
ts + offset

Timestamp('2018-01-08 10:00:00')

- These operations preserve time (hour, minute, etc) information by default. To reset time to midnight, use normalize() before or after applying the operation (depending on whether you want the time information included in the operation).

In [157]:
ts = pd.Timestamp("2014-01-01 09:00")
day = pd.offsets.Day()
day + ts

Timestamp('2014-01-02 09:00:00')

In [158]:
(day + ts).normalize()

Timestamp('2014-01-02 00:00:00')

In [159]:
ts = pd.Timestamp("2014-01-01 22:00")
hour = pd.offsets.Hour()
hour + ts


Timestamp('2014-01-01 23:00:00')

In [160]:
(hour + ts).normalize()

Timestamp('2014-01-01 00:00:00')

In [161]:
(hour + pd.Timestamp("2014-01-01 23:30")).normalize()

Timestamp('2014-01-02 00:00:00')

#### Parametric offsets
- Some of the offsets can be “parameterized” when created to result in different behaviors. For example, the Week offset for generating weekly data accepts a weekday parameter which results in the generated dates always lying on a particular day of the week:

In [162]:
d = datetime.datetime(2008, 8, 18, 9, 0)
d

datetime.datetime(2008, 8, 18, 9, 0)

In [163]:
d + pd.offsets.Week()

Timestamp('2008-08-25 09:00:00')

In [164]:
d + pd.offsets.Week(weekday=4)

Timestamp('2008-08-22 09:00:00')

In [165]:
(d + pd.offsets.Week(weekday=4)).weekday()

4

In [166]:
d - pd.offsets.Week()

Timestamp('2008-08-11 09:00:00')

- The normalize option will be effective for addition and subtraction.

In [167]:
d + pd.offsets.Week(normalize=True)

Timestamp('2008-08-25 00:00:00')

In [168]:
d - pd.offsets.Week(normalize=True)

Timestamp('2008-08-11 00:00:00')

- Another example is parameterizing YearEnd with the specific ending month:

In [169]:
d + pd.offsets.YearEnd()

Timestamp('2008-12-31 09:00:00')

In [170]:
d + pd.offsets.YearEnd(month=6)

Timestamp('2009-06-30 09:00:00')

#### Using offsets with Series / DatetimeIndex
- Offsets can be used with either a Series or DatetimeIndex to apply the offset to each element.

In [171]:
rng = pd.date_range("2012-01-01", "2012-01-03")
s = pd.Series(rng)
rng

DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03'], dtype='datetime64[ns]', freq='D')

In [173]:
rng + pd.DateOffset(months=2)

DatetimeIndex(['2012-03-01', '2012-03-02', '2012-03-03'], dtype='datetime64[ns]', freq=None)

In [174]:
s + pd.DateOffset(months=2)

0   2012-03-01
1   2012-03-02
2   2012-03-03
dtype: datetime64[ns]

In [175]:
s - pd.DateOffset(months=2)

0   2011-11-01
1   2011-11-02
2   2011-11-03
dtype: datetime64[ns]

- If the offset class maps directly to a Timedelta (Day, Hour, Minute, Second, Micro, Milli, Nano) it can be used exactly like a Timedelta - see the Timedelta section for more examples.

In [176]:
s - pd.offsets.Day(2)

0   2011-12-30
1   2011-12-31
2   2012-01-01
dtype: datetime64[ns]

In [177]:
td = s - pd.Series(pd.date_range("2011-12-29", "2011-12-31"))

td

0   3 days
1   3 days
2   3 days
dtype: timedelta64[ns]

In [178]:
td + pd.offsets.Minute(15)


0   3 days 00:15:00
1   3 days 00:15:00
2   3 days 00:15:00
dtype: timedelta64[ns]

- Note that some offsets (such as BQuarterEnd) do not have a vectorized implementation. They can still be used but may calculate significantly slower and will show a PerformanceWarning

In [179]:
rng + pd.offsets.BQuarterEnd()

DatetimeIndex(['2012-03-30', '2012-03-30', '2012-03-30'], dtype='datetime64[ns]', freq=None)

#### Custom business days
- The CDay or CustomBusinessDay class provides a parametric BusinessDay class which can be used to create customized business day calendars which account for local holidays and local weekend conventions.

- As an interesting example, let’s look at Egypt where a Friday-Saturday weekend is observed.

In [184]:
weekmask_egypt = "Sun Mon Tue Wed Thu"


# They also observe International Workers' Day so let's
# add that for a couple of years
holidays = [
    "2012-05-01",
    datetime.datetime(2013, 5, 1),
    np.datetime64("2014-05-01"),
]

bday_egypt = pd.offsets.CustomBusinessDay(
    holidays=holidays,
    weekmask=weekmask_egypt,
)

dt = datetime.datetime(2013, 4, 30)
dt + 2 * bday_egypt


Timestamp('2013-05-05 00:00:00')

- Let’s map to the weekday names:

In [185]:
dts = pd.date_range(dt, periods=5, freq=bday_egypt)
pd.Series(dts.weekday, dts).map(pd.Series("Mon Tue Wed Thu Fri Sat Sun".split()))


2013-04-30    Tue
2013-05-02    Thu
2013-05-05    Sun
2013-05-06    Mon
2013-05-07    Tue
Freq: C, dtype: object

- Holiday calendars can be used to provide the list of holidays. See the holiday calendar section for more information.

In [186]:
from pandas.tseries.holiday import USFederalHolidayCalendar
bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
# Friday before MLK Day
dt = datetime.datetime(2014, 1, 17)
# Tuesday after MLK Day (Monday is skipped because it's a holiday)
dt + bday_us

Timestamp('2014-01-21 00:00:00')

- Monthly offsets that respect a certain holiday calendar can be defined in the usual way.

In [187]:
bmth_us = pd.offsets.CustomBusinessMonthBegin(calendar=USFederalHolidayCalendar())
# Skip new years
dt = datetime.datetime(2013, 12, 17)
dt + bmth_us
# Define date index with custom offset
pd.date_range(start="20100101", end="20120101", freq=bmth_us)


DatetimeIndex(['2010-01-04', '2010-02-01', '2010-03-01', '2010-04-01',
               '2010-05-03', '2010-06-01', '2010-07-01', '2010-08-02',
               '2010-09-01', '2010-10-01', '2010-11-01', '2010-12-01',
               '2011-01-03', '2011-02-01', '2011-03-01', '2011-04-01',
               '2011-05-02', '2011-06-01', '2011-07-01', '2011-08-01',
               '2011-09-01', '2011-10-03', '2011-11-01', '2011-12-01'],
              dtype='datetime64[ns]', freq='CBMS')

- **Note**: The frequency string ‘C’ is used to indicate that a CustomBusinessDay DateOffset is used, it is important to note that since CustomBusinessDay is a parameterised type, instances of CustomBusinessDay may differ and this is not detectable from the ‘C’ frequency string. The user therefore needs to ensure that the ‘C’ frequency string is used consistently within the user’s application.

#### Business hour
- The BusinessHour class provides a business hour representation on BusinessDay, allowing to use specific start and end times.

- By default, BusinessHour uses 9:00 - 17:00 as business hours. Adding BusinessHour will increment Timestamp by hourly frequency. If target Timestamp is out of business hours, move to the next business hour then increment it. If the result exceeds the business hours end, the remaining hours are added to the next business day.

In [188]:
bh = pd.offsets.BusinessHour()
bh

<BusinessHour: bh=09:00-17:00>

In [189]:
# 2014-08-01 is Friday
pd.Timestamp("2014-08-01 10:00").weekday()

4

In [190]:
pd.Timestamp("2014-08-01 10:00") + bh

Timestamp('2014-08-01 11:00:00')

In [191]:
# Below example is the same as: pd.Timestamp('2014-08-01 09:00') + bh
pd.Timestamp("2014-08-01 08:00") + bh

Timestamp('2014-08-01 10:00:00')

In [192]:
# If the results is on the end time, move to the next business day
pd.Timestamp("2014-08-01 16:00") + bh

Timestamp('2014-08-04 09:00:00')

In [193]:
# Remainings are added to the next day
pd.Timestamp("2014-08-01 16:30") + bh

Timestamp('2014-08-04 09:30:00')

In [194]:
# Adding 2 business hours
pd.Timestamp("2014-08-01 10:00") + pd.offsets.BusinessHour(2)

Timestamp('2014-08-01 12:00:00')

In [195]:
# Subtracting 3 business hours
pd.Timestamp("2014-08-01 10:00") + pd.offsets.BusinessHour(-3)

Timestamp('2014-07-31 15:00:00')

- You can also specify start and end time by keywords. The argument must be a str with an hour:minute representation or a datetime.time instance. Specifying seconds, microseconds and nanoseconds as business hour results in ValueError.

In [196]:
bh = pd.offsets.BusinessHour(start="11:00", end=datetime.time(20, 0))
bh

<BusinessHour: bh=11:00-20:00>

In [197]:
pd.Timestamp("2014-08-01 13:00") + bh

Timestamp('2014-08-01 14:00:00')

In [198]:
pd.Timestamp("2014-08-01 09:00") + bh

Timestamp('2014-08-01 12:00:00')

In [199]:
pd.Timestamp("2014-08-01 18:00") + bh

Timestamp('2014-08-01 19:00:00')

- Passing start time later than end represents midnight business hour. In this case, business hour exceeds midnight and overlap to the next day. Valid business hours are distinguished by whether it started from valid BusinessDay.

In [200]:
bh = pd.offsets.BusinessHour(start="17:00", end="09:00")
bh

<BusinessHour: bh=17:00-09:00>

In [201]:
pd.Timestamp("2014-08-01 17:00") + bh

Timestamp('2014-08-01 18:00:00')

In [202]:
pd.Timestamp("2014-08-01 23:00") + bh

Timestamp('2014-08-02 00:00:00')

In [203]:
# Although 2014-08-02 is Saturday,
# it is valid because it starts from 08-01 (Friday).
pd.Timestamp("2014-08-02 04:00") + bh

Timestamp('2014-08-02 05:00:00')

In [204]:
# Although 2014-08-04 is Monday,
# it is out of business hours because it starts from 08-03 (Sunday).
pd.Timestamp("2014-08-04 04:00") + bh

Timestamp('2014-08-04 18:00:00')

- Applying BusinessHour.rollforward and rollback to out of business hours results in the next business hour start or previous day’s end. Different from other offsets, BusinessHour.rollforward may output different results from apply by definition.

- This is because one day’s business hour end is equal to next day’s business hour start. For example, under the default business hours (9:00 - 17:00), there is no gap (0 minutes) between 2014-08-01 17:00 and 2014-08-04 09:00.

In [205]:
# This adjusts a Timestamp to business hour edge
pd.offsets.BusinessHour().rollback(pd.Timestamp("2014-08-02 15:00"))

Timestamp('2014-08-01 17:00:00')

In [206]:
pd.offsets.BusinessHour().rollforward(pd.Timestamp("2014-08-02 15:00"))

Timestamp('2014-08-04 09:00:00')

In [207]:
# It is the same as BusinessHour() + pd.Timestamp('2014-08-01 17:00').
# And it is the same as BusinessHour() + pd.Timestamp('2014-08-04 09:00')
pd.offsets.BusinessHour() + pd.Timestamp("2014-08-02 15:00")

Timestamp('2014-08-04 10:00:00')

In [208]:
# BusinessDay results (for reference)
pd.offsets.BusinessHour().rollforward(pd.Timestamp("2014-08-02"))

Timestamp('2014-08-04 09:00:00')

In [209]:
# It is the same as BusinessDay() + pd.Timestamp('2014-08-01')
# The result is the same as rollworward because BusinessDay never overlap.
pd.offsets.BusinessHour() + pd.Timestamp("2014-08-02")

Timestamp('2014-08-04 10:00:00')

- BusinessHour regards Saturday and Sunday as holidays. To use arbitrary holidays, you can use CustomBusinessHour offset, as explained in the following subsection.

#### Custom business hour
- The CustomBusinessHour is a mixture of BusinessHour and CustomBusinessDay which allows you to specify arbitrary holidays. CustomBusinessHour works as the same as BusinessHour except that it skips specified custom holidays.

In [210]:
from pandas.tseries.holiday import USFederalHolidayCalendar

bhour_us = pd.offsets.CustomBusinessHour(calendar=USFederalHolidayCalendar())
# Friday before MLK Day
dt = datetime.datetime(2014, 1, 17, 15)
dt + bhour_us



Timestamp('2014-01-17 16:00:00')

In [211]:
# Tuesday after MLK Day (Monday is skipped because it's a holiday)
dt + bhour_us * 2

Timestamp('2014-01-21 09:00:00')

- You can use keyword arguments supported by either BusinessHour and CustomBusinessDay.

In [212]:
bhour_mon = pd.offsets.CustomBusinessHour(start="10:00", weekmask="Tue Wed Thu Fri")
# Monday is skipped because it's a holiday, business hour starts from 10:00
dt + bhour_mon * 2


Timestamp('2014-01-21 10:00:00')

#### Offset aliases
- A number of string aliases are given to useful common time series frequencies. We will refer to these aliases as offset aliases.

| Alias | Description |
| ---- | ----- |
| B | business day frequency |
| C | custom business day frequency |
| D | calendar day frequency |
| W | weekly frequency |
| ME | month end frequency |
| SME | semi-month end frequency (15th and end of month) |
| BME | business month end frequency |
| CBME | custom business month end frequency |
| MS | month start frequency |
| SMS | semi-month start frequency (1st and 15th) |
| BMS | business month start frequency |
| CBMS | custom business month start frequency |
| QE | quarter end frequency |
| BQE | business quarter end frequency |
| QS | quarter start frequency |
| BQS | business quarter start frequency |
| YE | year end frequency |
| BYE | business year end frequency |
| YS | year start frequency |
| BYS | business year start frequency |
| h | hourly frequency |
| bh | business hour frequency |
| cbh | custom business hour frequency |
| min | minutely frequency |
| s | secondly frequency  |
| ms | milliseconds |
| us | microseconds |
| ns | nanoseconds |

- **Note**: When using the offset aliases above, it should be noted that functions such as date_range(), bdate_range(), will only return timestamps that are in the interval defined by start_date and end_date. If the start_date does not correspond to the frequency, the returned timestamps will start at the next valid timestamp, same for end_date, the returned timestamps will stop at the previous valid timestamp.
- For example, for the offset MS, if the start_date is not the first of the month, the returned timestamps will start with the first day of the next month. If end_date is not the first day of a month, the last returned timestamp will be the first day of the corresponding month.

In [213]:
dates_lst_1 = pd.date_range("2020-01-06", "2020-04-03", freq="MS")
dates_lst_1

DatetimeIndex(['2020-02-01', '2020-03-01', '2020-04-01'], dtype='datetime64[ns]', freq='MS')

In [214]:
dates_lst_2 = pd.date_range("2020-01-01", "2020-04-01", freq="MS")
dates_lst_2

DatetimeIndex(['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01'], dtype='datetime64[ns]', freq='MS')

- We can see in the above example date_range() and bdate_range() will only return the valid timestamps between the start_date and end_date. If these are not valid timestamps for the given frequency it will roll to the next value for start_date (respectively previous for the end_date)

#### Period aliases
- A number of string aliases are given to useful common time series frequencies. We will refer to these aliases as period aliases.

| Alias | Description |
| ----- | ------ |
| B | business day frequency |
| D | calendar day frequency |
| W | weekly frequency |
| M | monthly frequency |
| Q | quarterly frequency |
| Y | yearly frequency |
| h | hourly frequency |
| min | minutely frequency |
| s | secondly frequency |
| ms | milliseconds |
| us |  microseconds |
| ns | nanoseconds |

#### Combining aliases
- As we have seen previously, the alias and the offset instance are fungible in most functions:

In [215]:
pd.date_range(start, periods=5, freq="B")

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07'],
              dtype='datetime64[ns]', freq='B')

In [216]:
pd.date_range(start, periods=5, freq=pd.offsets.BDay())

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07'],
              dtype='datetime64[ns]', freq='B')

- You can combine together day and intraday offsets:

In [217]:
pd.date_range(start, periods=10, freq="2h20min")

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 02:20:00',
               '2011-01-01 04:40:00', '2011-01-01 07:00:00',
               '2011-01-01 09:20:00', '2011-01-01 11:40:00',
               '2011-01-01 14:00:00', '2011-01-01 16:20:00',
               '2011-01-01 18:40:00', '2011-01-01 21:00:00'],
              dtype='datetime64[ns]', freq='140min')

In [218]:
pd.date_range(start, periods=10, freq="1D10us")

DatetimeIndex([       '2011-01-01 00:00:00', '2011-01-02 00:00:00.000010',
               '2011-01-03 00:00:00.000020', '2011-01-04 00:00:00.000030',
               '2011-01-05 00:00:00.000040', '2011-01-06 00:00:00.000050',
               '2011-01-07 00:00:00.000060', '2011-01-08 00:00:00.000070',
               '2011-01-09 00:00:00.000080', '2011-01-10 00:00:00.000090'],
              dtype='datetime64[ns]', freq='86400000010us')

#### Anchored offsets
- For some frequencies you can specify an anchoring suffix:

| Alias | Description |
| ----- | ----------- |
| W-SUN | weekly frequency (Sundays). Same as ‘W’ |
| W-MON | weekly frequency (Mondays) |
| W-TUE | weekly frequency (Tuesdays) |
| W-WED | weekly frequency (Wednesdays) |
| W-THU | weekly frequency (Thursdays) |
| W-FRI | weekly frequency (Fridays) |
| W-SAT | weekly frequency (Saturdays) |
| (B)Q(E)(S)-DEC | quarterly frequency, year ends in December. Same as ‘QE’ |
| (B)Q(E)(S)-JAN | quarterly frequency, year ends in January |
| (B)Q(E)(S)-FEB | quarterly frequency, year ends in February |
| (B)Q(E)(S)-MAR | quarterly frequency, year ends in March |
| (B)Q(E)(S)-APR | quarterly frequency, year ends in April |
| (B)Q(E)(S)-MAY | quarterly frequency, year ends in May |
| (B)Q(E)(S)-JUN | quarterly frequency, year ends in June |
| (B)Q(E)(S)-JUL | quarterly frequency, year ends in July |
| (B)Q(E)(S)-AUG | quarterly frequency, year ends in August |
| (B)Q(E)(S)-SEP | quarterly frequency, year ends in September |
| (B)Q(E)(S)-OCT | quarterly frequency, year ends in October |
| (B)Q(E)(S)-NOV | quarterly frequency, year ends in November |
| (B)Y(E)(S)-DEC | annual frequency, anchored end of December. Same as ‘YE’ |
| (B)Y(E)(S)-JAN | annual frequency, anchored end of January |
| (B)Y(E)(S)-FEB | annual frequency, anchored end of February |
| (B)Y(E)(S)-MAR | annual frequency, anchored end of March |
| (B)Y(E)(S)-APR | annual frequency, anchored end of April |
| (B)Y(E)(S)-MAY | annual frequency, anchored end of May |
| (B)Y(E)(S)-JUN | annual frequency, anchored end of June |
| (B)Y(E)(S)-JUL | annual frequency, anchored end of July |
| (B)Y(E)(S)-AUG | annual frequency, anchored end of August |
| (B)Y(E)(S)-SEP | annual frequency, anchored end of September |
| (B)Y(E)(S)-OCT | annual frequency, anchored end of October |
| (B)Y(E)(S)-NOV | annual frequency, anchored end of November |

- These can be used as arguments to date_range, bdate_range, constructors for DatetimeIndex, as well as various other timeseries-related functions in pandas.

#### Anchored offset semantics
- For those offsets that are anchored to the start or end of specific frequency (MonthEnd, MonthBegin, WeekEnd, etc), the following rules apply to rolling forward and backwards.

- When n is not 0, if the given date is not on an anchor point, it snapped to the next(previous) anchor point, and moved |n|-1 additional steps forwards or backwards.

In [219]:
pd.Timestamp("2014-01-02") + pd.offsets.MonthBegin(n=1)

Timestamp('2014-02-01 00:00:00')

In [220]:
pd.Timestamp("2014-01-02") + pd.offsets.MonthEnd(n=1)

Timestamp('2014-01-31 00:00:00')

In [221]:
pd.Timestamp("2014-01-02") - pd.offsets.MonthBegin(n=1)

Timestamp('2014-01-01 00:00:00')

In [222]:
pd.Timestamp("2014-01-02") - pd.offsets.MonthEnd(n=1)

Timestamp('2013-12-31 00:00:00')

In [223]:
pd.Timestamp("2014-01-02") + pd.offsets.MonthBegin(n=4)

Timestamp('2014-05-01 00:00:00')

In [224]:
pd.Timestamp("2014-01-02") - pd.offsets.MonthBegin(n=4)

Timestamp('2013-10-01 00:00:00')

- If the given date is on an anchor point, it is moved |n| points forwards or backwards.

In [225]:
pd.Timestamp("2014-01-01") + pd.offsets.MonthBegin(n=1)

Timestamp('2014-02-01 00:00:00')

In [226]:
pd.Timestamp("2014-01-31") + pd.offsets.MonthEnd(n=1)

Timestamp('2014-02-28 00:00:00')

In [227]:
pd.Timestamp("2014-01-01") - pd.offsets.MonthBegin(n=1)

Timestamp('2013-12-01 00:00:00')

In [228]:
pd.Timestamp("2014-01-31") - pd.offsets.MonthEnd(n=1)

Timestamp('2013-12-31 00:00:00')

In [229]:
pd.Timestamp("2014-01-01") + pd.offsets.MonthBegin(n=4)

Timestamp('2014-05-01 00:00:00')

In [230]:
pd.Timestamp("2014-01-31") - pd.offsets.MonthBegin(n=4)

Timestamp('2013-10-01 00:00:00')

- For the case when n=0, the date is not moved if on an anchor point, otherwise it is rolled forward to the next anchor point.

In [231]:
pd.Timestamp("2014-01-02") + pd.offsets.MonthBegin(n=0)

Timestamp('2014-02-01 00:00:00')

In [232]:
pd.Timestamp("2014-01-02") + pd.offsets.MonthEnd(n=0)

Timestamp('2014-01-31 00:00:00')

In [233]:
pd.Timestamp("2014-01-01") + pd.offsets.MonthBegin(n=0)

Timestamp('2014-01-01 00:00:00')

In [234]:
pd.Timestamp("2014-01-31") + pd.offsets.MonthEnd(n=0)

Timestamp('2014-01-31 00:00:00')

#### Holidays / holiday calendars
- Holidays and calendars provide a simple way to define holiday rules to be used with CustomBusinessDay or in other analysis that requires a predefined set of holidays. The AbstractHolidayCalendar class provides all the necessary methods to return a list of holidays and only rules need to be defined in a specific holiday calendar class. Furthermore, the start_date and end_date class attributes determine over what date range holidays are generated. These should be overwritten on the AbstractHolidayCalendar class to have the range apply to all calendar subclasses. USFederalHolidayCalendar is the only calendar that exists and primarily serves as an example for developing other calendars.

- For holidays that occur on fixed dates (e.g., US Memorial Day or July 4th) an observance rule determines when that holiday is observed if it falls on a weekend or some other non-observed day. Defined observance rules are:

| Rule | Description |
| ----- | -------- |
| nearest_workday | move Saturday to Friday and Sunday to Monday |
| sunday_to_monday | move Sunday to following Monday |
| next_monday_or_tuesday | move Saturday to Monday and Sunday/Monday to Tuesday |
| previous_friday | move Saturday and Sunday to previous Friday” |
| next_monday | move Saturday and Sunday to following Monday |

- An example of how holidays and holiday calendars are defined:

In [235]:
from pandas.tseries.holiday import (
    Holiday,
    USMemorialDay,
    AbstractHolidayCalendar,
    nearest_workday,
    MO,
)

class ExampleCalendar(AbstractHolidayCalendar):
    rules = [
        USMemorialDay,
        Holiday("July 4th", month=7, day=4, observance=nearest_workday),
        Holiday(
            "Columbus Day",
            month=10,
            day=1,
            offset=pd.DateOffset(weekday=MO(2)),
        ),
    ]

cal = ExampleCalendar()

cal.holidays(datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31))


DatetimeIndex(['2012-05-28', '2012-07-04', '2012-10-08'], dtype='datetime64[ns]', freq=None)

- **hint**: weekday=MO(2) is same as 2 * Week(weekday=2)

- Using this calendar, creating an index or doing offset arithmetic skips weekends and holidays (i.e., Memorial Day/July 4th). For example, the below defines a custom business day offset using the ExampleCalendar. Like any other offset, it can be used to create a DatetimeIndex or added to datetime or Timestamp objects.

In [236]:
pd.date_range(
    start="7/1/2012", end="7/10/2012", freq=pd.offsets.CDay(calendar=cal)
).to_pydatetime()

array([datetime.datetime(2012, 7, 2, 0, 0),
       datetime.datetime(2012, 7, 3, 0, 0),
       datetime.datetime(2012, 7, 5, 0, 0),
       datetime.datetime(2012, 7, 6, 0, 0),
       datetime.datetime(2012, 7, 9, 0, 0),
       datetime.datetime(2012, 7, 10, 0, 0)], dtype=object)

In [237]:
offset = pd.offsets.CustomBusinessDay(calendar=cal)
datetime.datetime(2012, 5, 25) + offset


Timestamp('2012-05-29 00:00:00')

In [238]:
datetime.datetime(2012, 7, 3) + offset

Timestamp('2012-07-05 00:00:00')

In [239]:
datetime.datetime(2012, 7, 3) + 2 * offset

Timestamp('2012-07-06 00:00:00')

In [240]:
datetime.datetime(2012, 7, 6) + offset

Timestamp('2012-07-09 00:00:00')

- Ranges are defined by the start_date and end_date class attributes of AbstractHolidayCalendar. The defaults are shown below.

In [241]:
AbstractHolidayCalendar.start_date

Timestamp('1970-01-01 00:00:00')

In [242]:
AbstractHolidayCalendar.end_date

Timestamp('2200-12-31 00:00:00')

- These dates can be overwritten by setting the attributes as datetime/Timestamp/string.

In [243]:
AbstractHolidayCalendar.start_date = datetime.datetime(2012, 1, 1)
AbstractHolidayCalendar.end_date = datetime.datetime(2012, 12, 31)
cal.holidays()

DatetimeIndex(['2012-05-28', '2012-07-04', '2012-10-08'], dtype='datetime64[ns]', freq=None)

- Every calendar class is accessible by name using the get_calendar function which returns a holiday class instance. Any imported calendar class will automatically be available by this function. Also, HolidayCalendarFactory provides an easy interface to create calendars that are combinations of calendars or calendars with additional rules.

In [244]:
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, USLaborDay

cal = get_calendar("ExampleCalendar")
cal.rules

[Holiday: Memorial Day (month=5, day=31, offset=<DateOffset: weekday=MO(-1)>),
 Holiday: July 4th (month=7, day=4, observance=<function nearest_workday at 0x000001DBF1FA3600>),
 Holiday: Columbus Day (month=10, day=1, offset=<DateOffset: weekday=MO(+2)>)]

In [245]:
new_cal = HolidayCalendarFactory("NewExampleCalendar", cal, USLaborDay)
new_cal.rules

[Holiday: Labor Day (month=9, day=1, offset=<DateOffset: weekday=MO(+1)>),
 Holiday: Memorial Day (month=5, day=31, offset=<DateOffset: weekday=MO(-1)>),
 Holiday: July 4th (month=7, day=4, observance=<function nearest_workday at 0x000001DBF1FA3600>),
 Holiday: Columbus Day (month=10, day=1, offset=<DateOffset: weekday=MO(+2)>)]

#### Time Series-related instance methods
#### Shifting / lagging
- One may want to shift or lag the values in a time series back and forward in time. The method for this is shift(), which is available on all of the pandas objects.

In [246]:
ts = pd.Series(range(len(rng)), index=rng)
ts = ts[:5]
ts.shift(1)

2012-01-01    NaN
2012-01-02    0.0
2012-01-03    1.0
Freq: D, dtype: float64

- The shift method accepts an freq argument which can accept a DateOffset class or other timedelta-like object or also an offset alias.

- When freq is specified, shift method changes all the dates in the index rather than changing the alignment of the data and the index:

In [247]:
ts.shift(5, freq="D")

2012-01-06    0
2012-01-07    1
2012-01-08    2
Freq: D, dtype: int64

In [248]:
ts.shift(5, freq=pd.offsets.BDay())

2012-01-06    0
2012-01-09    1
2012-01-10    2
dtype: int64

In [249]:
ts.shift(5, freq="BME")

2012-05-31    0
2012-05-31    1
2012-05-31    2
dtype: int64

- **Note** that with when freq is specified, the leading entry is no longer NaN because the data is not being realigned.

#### Frequency conversion
- The primary function for changing frequencies is the asfreq() method. For a DatetimeIndex, this is basically just a thin, but convenient wrapper around reindex() which generates a date_range and calls reindex.

In [250]:
dr = pd.date_range("1/1/2010", periods=3, freq=3 * pd.offsets.BDay())
ts = pd.Series(np.random.randn(3), index=dr)
ts

2010-01-01   -1.344358
2010-01-06    0.117293
2010-01-11    1.168115
Freq: 3B, dtype: float64

In [251]:
ts.asfreq(pd.offsets.BDay())

2010-01-01   -1.344358
2010-01-04         NaN
2010-01-05         NaN
2010-01-06    0.117293
2010-01-07         NaN
2010-01-08         NaN
2010-01-11    1.168115
Freq: B, dtype: float64

- asfreq provides a further convenience so you can specify an interpolation method for any gaps that may appear after the frequency conversion.

In [252]:
ts.asfreq(pd.offsets.BDay(), method="pad")

2010-01-01   -1.344358
2010-01-04   -1.344358
2010-01-05   -1.344358
2010-01-06    0.117293
2010-01-07    0.117293
2010-01-08    0.117293
2010-01-11    1.168115
Freq: B, dtype: float64

#### Filling forward / backward
- Related to asfreq and reindex is fillna(), which is documented in the missing data section.

#### Converting to Python datetimes
- DatetimeIndex can be converted to an array of Python native datetime.datetime objects using the to_pydatetime method.

#### Resampling
- pandas has a simple, powerful, and efficient functionality for performing resampling operations during frequency conversion (e.g., converting secondly data into 5-minutely data). This is extremely common in, but not limited to, financial applications.

- resample() is a time-based groupby, followed by a reduction method on each of its groups. See some cookbook examples for some advanced strategies.

- The resample() method can be used directly from DataFrameGroupBy objects, see the groupby docs.

#### Basics

In [253]:
rng = pd.date_range("1/1/2012", periods=100, freq="s")
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
ts.resample("5Min").sum()

2012-01-01    24492
Freq: 5min, dtype: int32

- The resample function is very flexible and allows you to specify many different parameters to control the frequency conversion and resampling operation.

- Any built-in method available via GroupBy is available as a method of the returned object, including sum, mean, std, sem, max, min, median, first, last, ohlc:

In [254]:
ts.resample("5Min").mean()


2012-01-01    244.92
Freq: 5min, dtype: float64

In [255]:
ts.resample("5Min").ohlc()

,open,high,low,close
2012-01-01,102,493,0,314


In [256]:
ts.resample("5Min").max()

2012-01-01    493
Freq: 5min, dtype: int32

- For downsampling, closed can be set to ‘left’ or ‘right’ to specify which end of the interval is closed:

In [257]:
ts.resample("5Min", closed="right").mean()

2011-12-31 23:55:00    102.000000
2012-01-01 00:00:00    246.363636
Freq: 5min, dtype: float64

In [258]:
ts.resample("5Min", closed="left").mean()

2012-01-01    244.92
Freq: 5min, dtype: float64

- Parameters like label are used to manipulate the resulting labels. label specifies whether the result is labeled with the beginning or the end of the interval.

In [259]:
ts.resample("5Min").mean()  # by default label='left'

2012-01-01    244.92
Freq: 5min, dtype: float64

In [260]:
ts.resample("5Min", label="left").mean()

2012-01-01    244.92
Freq: 5min, dtype: float64

- **Warning**: The default values for label and closed is ‘left’ for all frequency offsets except for ‘ME’, ‘YE’, ‘QE’, ‘BME’, ‘BYE’, ‘BQE’, and ‘W’ which all have a default of ‘right’.

- This might unintendedly lead to looking ahead, where the value for a later time is pulled back to a previous time as in the following example with the BusinessDay frequency:

In [261]:
s = pd.date_range("2000-01-01", "2000-01-05").to_series()
s.iloc[2] = pd.NaT
s.dt.day_name()

2000-01-01     Saturday
2000-01-02       Sunday
2000-01-03          NaN
2000-01-04      Tuesday
2000-01-05    Wednesday
Freq: D, dtype: object

In [262]:
# default: label='left', closed='left'
s.resample("B").last().dt.day_name()

1999-12-31       Sunday
2000-01-03          NaN
2000-01-04      Tuesday
2000-01-05    Wednesday
Freq: B, dtype: object

- Notice how the value for Sunday got pulled back to the previous Friday. To get the behavior where the value for Sunday is pushed to Monday, use instead

In [263]:
s.resample("B", label="right", closed="right").last().dt.day_name()


2000-01-03       Sunday
2000-01-04      Tuesday
2000-01-05    Wednesday
2000-01-06          NaN
Freq: B, dtype: object

- The axis parameter can be set to 0 or 1 and allows you to resample the specified axis for a DataFrame.

- kind can be set to ‘timestamp’ or ‘period’ to convert the resulting index to/from timestamp and time span representations. By default resample retains the input representation.

- convention can be set to ‘start’ or ‘end’ when resampling period data (detail below). It specifies how low frequency periods are converted to higher frequency periods.

#### Upsampling
- For upsampling, you can specify a way to upsample and the limit parameter to interpolate over the gaps that are created:

In [264]:
# from secondly to every 250 milliseconds
ts[:2].resample("250ms").asfreq()

2012-01-01 00:00:00.000    102.0
2012-01-01 00:00:00.250      NaN
2012-01-01 00:00:00.500      NaN
2012-01-01 00:00:00.750      NaN
2012-01-01 00:00:01.000    127.0
Freq: 250ms, dtype: float64

In [265]:
ts[:2].resample("250ms").ffill()

2012-01-01 00:00:00.000    102
2012-01-01 00:00:00.250    102
2012-01-01 00:00:00.500    102
2012-01-01 00:00:00.750    102
2012-01-01 00:00:01.000    127
Freq: 250ms, dtype: int32

In [266]:
ts[:2].resample("250ms").ffill(limit=2)

2012-01-01 00:00:00.000    102.0
2012-01-01 00:00:00.250    102.0
2012-01-01 00:00:00.500    102.0
2012-01-01 00:00:00.750      NaN
2012-01-01 00:00:01.000    127.0
Freq: 250ms, dtype: float64

#### Sparse resampling
- Sparse timeseries are the ones where you have a lot fewer points relative to the amount of time you are looking to resample. Naively upsampling a sparse series can potentially generate lots of intermediate values. When you don’t want to use a method to fill these values, e.g. fill_method is None, then intermediate values will be filled with NaN.

- Since resample is a time-based groupby, the following is a method to efficiently resample only the groups that are not all NaN.

In [267]:
rng = pd.date_range("2014-1-1", periods=100, freq="D") + pd.Timedelta("1s")
ts = pd.Series(range(100), index=rng)

- If we want to resample to the full range of the series:

In [268]:
ts.resample("3min").sum()

2014-01-01 00:00:00     0
2014-01-01 00:03:00     0
2014-01-01 00:06:00     0
2014-01-01 00:09:00     0
2014-01-01 00:12:00     0
                       ..
2014-04-09 23:48:00     0
2014-04-09 23:51:00     0
2014-04-09 23:54:00     0
2014-04-09 23:57:00     0
2014-04-10 00:00:00    99
Freq: 3min, Length: 47521, dtype: int64

- We can instead only resample those groups where we have points as follows:

In [269]:
from functools import partial
from pandas.tseries.frequencies import to_offset

def round(t, freq):
    freq = to_offset(freq)
    td = pd.Timedelta(freq)
    return pd.Timestamp((t.value // td.value) * td.value)

ts.groupby(partial(round, freq="3min")).sum()

2014-01-01     0
2014-01-02     1
2014-01-03     2
2014-01-04     3
2014-01-05     4
              ..
2014-04-06    95
2014-04-07    96
2014-04-08    97
2014-04-09    98
2014-04-10    99
Length: 100, dtype: int64

#### Aggregation
- The resample() method returns a pandas.api.typing.Resampler instance. Similar to the aggregating API, groupby API, and the window API, a Resampler can be selectively resampled.

- Resampling a DataFrame, the default will be to act on all columns with the same function.

In [270]:
df = pd.DataFrame(
    np.random.randn(1000, 3),
    index=pd.date_range("1/1/2012", freq="s", periods=1000),
    columns=["A", "B", "C"],
)
r = df.resample("3min")
r.mean()

,A,B,C
2012-01-01 00:00:00,-0.064299,-0.086650,0.035856
2012-01-01 00:03:00,0.034916,0.031799,0.040840
2012-01-01 00:06:00,0.006057,-0.146410,0.056525
2012-01-01 00:09:00,-0.064865,0.069545,0.105333
2012-01-01 00:12:00,-0.101904,-0.092844,0.013785
2012-01-01 00:15:00,0.037054,0.012322,-0.057477


- We can select a specific column or columns using standard getitem.

In [271]:
r["A"].mean()

2012-01-01 00:00:00   -0.064299
2012-01-01 00:03:00    0.034916
2012-01-01 00:06:00    0.006057
2012-01-01 00:09:00   -0.064865
2012-01-01 00:12:00   -0.101904
2012-01-01 00:15:00    0.037054
Freq: 3min, Name: A, dtype: float64

In [272]:
r[["A", "B"]].mean()

,A,B
2012-01-01 00:00:00,-0.064299,-0.086650
2012-01-01 00:03:00,0.034916,0.031799
2012-01-01 00:06:00,0.006057,-0.146410
2012-01-01 00:09:00,-0.064865,0.069545
2012-01-01 00:12:00,-0.101904,-0.092844
2012-01-01 00:15:00,0.037054,0.012322


- You can pass a list or dict of functions to do aggregation with, outputting a DataFrame:

In [273]:
r["A"].agg(["sum", "mean", "std"])

,sum,mean,std
2012-01-01 00:00:00,-11.573823,-0.064299,0.939240
2012-01-01 00:03:00,6.284858,0.034916,0.946683
2012-01-01 00:06:00,1.090172,0.006057,0.966708
2012-01-01 00:09:00,-11.675672,-0.064865,0.955326
2012-01-01 00:12:00,-18.342684,-0.101904,0.885761
2012-01-01 00:15:00,3.705426,0.037054,0.877545


- On a resampled DataFrame, you can pass a list of functions to apply to each column, which produces an aggregated result with a hierarchical index:

In [274]:
r.agg(["sum", "mean"])

A                    B                    C  \
                           sum      mean        sum      mean        sum   
2012-01-01 00:00:00 -11.573823 -0.064299 -15.596999 -0.086650   6.454038   
2012-01-01 00:03:00   6.284858  0.034916   5.723804  0.031799   7.351267   
2012-01-01 00:06:00   1.090172  0.006057 -26.353875 -0.146410  10.174469   
2012-01-01 00:09:00 -11.675672 -0.064865  12.518029  0.069545  18.959997   
2012-01-01 00:12:00 -18.342684 -0.101904 -16.711850 -0.092844   2.481241   
2012-01-01 00:15:00   3.705426  0.037054   1.232212  0.012322  -5.747717   

                               
                         mean  
2012-01-01 00:00:00  0.035856  
2012-01-01 00:03:00  0.040840  
2012-01-01 00:06:00  0.056525  
2012-01-01 00:09:00  0.105333  
2012-01-01 00:12:00  0.013785  
2012-01-01 00:15:00 -0.057477

- By passing a dict to aggregate you can apply a different aggregation to the columns of a DataFrame:

In [275]:
r.agg({"A": "sum", "B": lambda x: np.std(x, ddof=1)})


,A,B
2012-01-01 00:00:00,-11.573823,1.028164
2012-01-01 00:03:00,6.284858,1.013792
2012-01-01 00:06:00,1.090172,0.951346
2012-01-01 00:09:00,-11.675672,0.996675
2012-01-01 00:12:00,-18.342684,1.042891
2012-01-01 00:15:00,3.705426,0.963981


- The function names can also be strings. In order for a string to be valid it must be implemented on the resampled object:

In [276]:
r.agg({"A": "sum", "B": "std"})

,A,B
2012-01-01 00:00:00,-11.573823,1.028164
2012-01-01 00:03:00,6.284858,1.013792
2012-01-01 00:06:00,1.090172,0.951346
2012-01-01 00:09:00,-11.675672,0.996675
2012-01-01 00:12:00,-18.342684,1.042891
2012-01-01 00:15:00,3.705426,0.963981


- Furthermore, you can also specify multiple aggregation functions for each column separately.

In [277]:
r.agg({"A": ["sum", "std"], "B": ["mean", "std"]})

A                   B          
                           sum       std      mean       std
2012-01-01 00:00:00 -11.573823  0.939240 -0.086650  1.028164
2012-01-01 00:03:00   6.284858  0.946683  0.031799  1.013792
2012-01-01 00:06:00   1.090172  0.966708 -0.146410  0.951346
2012-01-01 00:09:00 -11.675672  0.955326  0.069545  0.996675
2012-01-01 00:12:00 -18.342684  0.885761 -0.092844  1.042891
2012-01-01 00:15:00   3.705426  0.877545  0.012322  0.963981

- If a DataFrame does not have a datetimelike index, but instead you want to resample based on datetimelike column in the frame, it can passed to the on keyword.

In [278]:
df = pd.DataFrame(
    {"date": pd.date_range("2015-01-01", freq="W", periods=5), "a": np.arange(5)},
    index=pd.MultiIndex.from_arrays(
        [[1, 2, 3, 4, 5], pd.date_range("2015-01-01", freq="W", periods=5)],
        names=["v", "d"],
    ),
)
df

,,date,a
v,d,,
1,2015-01-04,2015-01-04,0
2,2015-01-11,2015-01-11,1
3,2015-01-18,2015-01-18,2
4,2015-01-25,2015-01-25,3
5,2015-02-01,2015-02-01,4


In [279]:
df.resample("ME", on="date")[["a"]].sum()

,a
date,
2015-01-31,6
2015-02-28,4


- Similarly, if you instead want to resample by a datetimelike level of MultiIndex, its name or location can be passed to the level keyword.

In [280]:
df.resample("ME", level="d")[["a"]].sum()

,a
d,
2015-01-31,6
2015-02-28,4


#### Iterating through groups
- With the Resampler object in hand, iterating through the grouped data is very natural and functions similarly to itertools.groupby():

In [281]:
small = pd.Series(
    range(6),
    index=pd.to_datetime(
        [
            "2017-01-01T00:00:00",
            "2017-01-01T00:30:00",
            "2017-01-01T00:31:00",
            "2017-01-01T01:00:00",
            "2017-01-01T03:00:00",
            "2017-01-01T03:05:00",
        ]
    ),
)
resampled = small.resample("h")
for name, group in resampled:
    print("Group: ", name)
    print("-" * 27)
    print(group, end="\n\n")

Group:  2017-01-01 00:00:00
---------------------------
2017-01-01 00:00:00    0
2017-01-01 00:30:00    1
2017-01-01 00:31:00    2
dtype: int64

Group:  2017-01-01 01:00:00
---------------------------
2017-01-01 01:00:00    3
dtype: int64

Group:  2017-01-01 02:00:00
---------------------------
Series([], dtype: int64)

Group:  2017-01-01 03:00:00
---------------------------
2017-01-01 03:00:00    4
2017-01-01 03:05:00    5
dtype: int64



#### Use origin or offset to adjust the start of the bins
- The bins of the grouping are adjusted based on the beginning of the day of the time series starting point. This works well with frequencies that are multiples of a day (like 30D) or that divide a day evenly (like 90s or 1min). This can create inconsistencies with some frequencies that do not meet this criteria. To change this behavior you can specify a fixed Timestamp with the argument origin.

- For example:

In [282]:
start, end = "2000-10-01 23:30:00", "2000-10-02 00:30:00"
middle = "2000-10-02 00:00:00"
rng = pd.date_range(start, end, freq="7min")
ts = pd.Series(np.arange(len(rng)) * 3, index=rng)
ts

2000-10-01 23:30:00     0
2000-10-01 23:37:00     3
2000-10-01 23:44:00     6
2000-10-01 23:51:00     9
2000-10-01 23:58:00    12
2000-10-02 00:05:00    15
2000-10-02 00:12:00    18
2000-10-02 00:19:00    21
2000-10-02 00:26:00    24
Freq: 7min, dtype: int64

- Here we can see that, when using origin with its default value ('start_day'), the result after '2000-10-02 00:00:00' are not identical depending on the start of time series:

In [283]:
ts.resample("17min", origin="start_day").sum()

2000-10-01 23:14:00     0
2000-10-01 23:31:00     9
2000-10-01 23:48:00    21
2000-10-02 00:05:00    54
2000-10-02 00:22:00    24
Freq: 17min, dtype: int64

In [284]:
ts[middle:end].resample("17min", origin="start_day").sum()

2000-10-02 00:00:00    33
2000-10-02 00:17:00    45
Freq: 17min, dtype: int64

- Here we can see that, when setting origin to 'epoch', the result after '2000-10-02 00:00:00' are identical depending on the start of time series:

In [285]:
ts.resample("17min", origin="epoch").sum()

2000-10-01 23:18:00     0
2000-10-01 23:35:00    18
2000-10-01 23:52:00    27
2000-10-02 00:09:00    39
2000-10-02 00:26:00    24
Freq: 17min, dtype: int64

In [286]:
ts[middle:end].resample("17min", origin="epoch").sum()

2000-10-01 23:52:00    15
2000-10-02 00:09:00    39
2000-10-02 00:26:00    24
Freq: 17min, dtype: int64

- If needed you can use a custom timestamp for origin:

In [287]:
ts.resample("17min", origin="2001-01-01").sum()

2000-10-01 23:30:00     9
2000-10-01 23:47:00    21
2000-10-02 00:04:00    54
2000-10-02 00:21:00    24
Freq: 17min, dtype: int64

In [288]:
ts[middle:end].resample("17min", origin=pd.Timestamp("2001-01-01")).sum()


2000-10-02 00:04:00    54
2000-10-02 00:21:00    24
Freq: 17min, dtype: int64

- If needed you can just adjust the bins with an offset Timedelta that would be added to the default origin. Those two examples are equivalent for this time series:

In [289]:
ts.resample("17min", origin="start").sum()

2000-10-01 23:30:00     9
2000-10-01 23:47:00    21
2000-10-02 00:04:00    54
2000-10-02 00:21:00    24
Freq: 17min, dtype: int64

In [290]:
ts.resample("17min", offset="23h30min").sum()

2000-10-01 23:30:00     9
2000-10-01 23:47:00    21
2000-10-02 00:04:00    54
2000-10-02 00:21:00    24
Freq: 17min, dtype: int64

- Note the use of 'start' for origin on the last example. In that case, origin will be set to the first value of the timeseries.

#### Backward resample
- Instead of adjusting the beginning of bins, sometimes we need to fix the end of the bins to make a backward resample with a given freq. The backward resample sets closed to 'right' by default since the last value should be considered as the edge point for the last bin.

- We can set origin to 'end'. The value for a specific Timestamp index stands for the resample result from the current Timestamp minus freq to the current Timestamp with a right close.

In [291]:
ts.resample('17min', origin='end').sum()

2000-10-01 23:35:00     0
2000-10-01 23:52:00    18
2000-10-02 00:09:00    27
2000-10-02 00:26:00    63
Freq: 17min, dtype: int64

- Besides, in contrast with the 'start_day' option, end_day is supported. This will set the origin as the ceiling midnight of the largest Timestamp.

In [292]:
ts.resample('17min', origin='end_day').sum()

2000-10-01 23:38:00     3
2000-10-01 23:55:00    15
2000-10-02 00:12:00    45
2000-10-02 00:29:00    45
Freq: 17min, dtype: int64

- The above result uses 2000-10-02 00:29:00 as the last bin’s right edge since the following computation.

In [293]:
ceil_mid = rng.max().ceil('D')
freq = pd.offsets.Minute(17)
bin_res = ceil_mid - freq * ((ceil_mid - rng.max()) // freq)
bin_res


Timestamp('2000-10-02 00:29:00')

#### Time span representation
- Regular intervals of time are represented by Period objects in pandas while sequences of Period objects are collected in a PeriodIndex, which can be created with the convenience function period_range.

#### Period
- A Period represents a span of time (e.g., a day, a month, a quarter, etc). You can specify the span via freq keyword using a frequency alias like below. Because freq represents a span of Period, it cannot be negative like “-3D”.

In [294]:
pd.Period("2012", freq="Y-DEC")

Period('2012', 'Y-DEC')

In [295]:
pd.Period("2012-1-1", freq="D")

Period('2012-01-01', 'D')

In [296]:
pd.Period("2012-1-1 19:00", freq="h")

Period('2012-01-01 19:00', 'h')

In [297]:
pd.Period("2012-1-1 19:00", freq="5h")

Period('2012-01-01 19:00', '5h')

- Adding and subtracting integers from periods shifts the period by its own frequency. Arithmetic is not allowed between Period with different freq (span).

In [298]:
p = pd.Period("2012", freq="Y-DEC")
p + 1

Period('2013', 'Y-DEC')

In [299]:
p - 3

Period('2009', 'Y-DEC')

In [300]:
p = pd.Period("2012-01", freq="2M")
p + 2

Period('2012-05', '2M')

In [301]:
p - 1

Period('2011-11', '2M')

In [302]:
p == pd.Period("2012-01", freq="3M")

False

- If Period freq is daily or higher (D, h, min, s, ms, us, and ns), offsets and timedelta-like can be added if the result can have the same freq. Otherwise, ValueError will be raised.

In [303]:
p = pd.Period("2014-07-01 09:00", freq="h")
p + pd.offsets.Hour(2)

Period('2014-07-01 11:00', 'h')

In [304]:
p + datetime.timedelta(minutes=120)

Period('2014-07-01 11:00', 'h')

In [305]:
p + np.timedelta64(7200, "s")

Period('2014-07-01 11:00', 'h')

In [306]:
try:
    p + pd.offsets.Minute(5)
except ValueError as e:
    print(e)

Input cannot be converted to Period(freq=h)


- If Period has other frequencies, only the same offsets can be added. Otherwise, ValueError will be raised.

In [307]:
p = pd.Period("2014-07", freq="M")
p + pd.offsets.MonthEnd(3)

Period('2014-10', 'M')

In [308]:
try:
    p + pd.offsets.MonthBegin(3)
except Exception as e:
    print(e)

Input has different freq=3M from Period(freq=M)


- Taking the difference of Period instances with the same frequency will return the number of frequency units between them:

In [309]:
pd.Period("2012", freq="Y-DEC") - pd.Period("2002", freq="Y-DEC")

<10 * YearEnds: month=12>

#### PeriodIndex and period_range
- Regular sequences of Period objects can be collected in a PeriodIndex, which can be constructed using the period_range convenience function:

In [310]:
prng = pd.period_range("1/1/2011", "1/1/2012", freq="M")
prng

PeriodIndex(['2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
             '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
             '2012-01'],
            dtype='period[M]')

- The PeriodIndex constructor can also be used directly:

In [311]:
pd.PeriodIndex(["2011-1", "2011-2", "2011-3"], freq="M")

PeriodIndex(['2011-01', '2011-02', '2011-03'], dtype='period[M]')

- Passing multiplied frequency outputs a sequence of Period which has multiplied span.

In [312]:
pd.period_range(start="2014-01", freq="3M", periods=4)

PeriodIndex(['2014-01', '2014-04', '2014-07', '2014-10'], dtype='period[3M]')

- If start or end are Period objects, they will be used as anchor endpoints for a PeriodIndex with frequency matching that of the PeriodIndex constructor.

In [313]:
pd.period_range(
    start=pd.Period("2017Q1", freq="Q"), end=pd.Period("2017Q2", freq="Q"), freq="M"
)

PeriodIndex(['2017-03', '2017-04', '2017-05', '2017-06'], dtype='period[M]')

- Just like DatetimeIndex, a PeriodIndex can also be used to index pandas objects:

In [314]:
ps = pd.Series(np.random.randn(len(prng)), prng)
ps

2011-01    0.802178
2011-02    1.037438
2011-03    0.032244
2011-04    0.136508
2011-05   -0.833892
2011-06    1.204212
2011-07   -1.083029
2011-08   -0.156050
2011-09   -0.447367
2011-10    0.477741
2011-11    1.711656
2011-12   -0.911262
2012-01    0.784511
Freq: M, dtype: float64

- PeriodIndex supports addition and subtraction with the same rule as Period.

In [315]:
idx = pd.period_range("2014-07-01 09:00", periods=5, freq="h")
idx

PeriodIndex(['2014-07-01 09:00', '2014-07-01 10:00', '2014-07-01 11:00',
             '2014-07-01 12:00', '2014-07-01 13:00'],
            dtype='period[h]')

In [316]:
idx + pd.offsets.Hour(2)

PeriodIndex(['2014-07-01 11:00', '2014-07-01 12:00', '2014-07-01 13:00',
             '2014-07-01 14:00', '2014-07-01 15:00'],
            dtype='period[h]')

In [317]:
idx = pd.period_range("2014-07", periods=5, freq="M")
idx

PeriodIndex(['2014-07', '2014-08', '2014-09', '2014-10', '2014-11'], dtype='period[M]')

In [318]:
idx + pd.offsets.MonthEnd(3)

PeriodIndex(['2014-10', '2014-11', '2014-12', '2015-01', '2015-02'], dtype='period[M]')

- PeriodIndex has its own dtype named period, refer to Period Dtypes.

#### Period dtypes
- PeriodIndex has a custom period dtype. This is a pandas extension dtype similar to the timezone aware dtype (datetime64[ns, tz]).

- The period dtype holds the freq attribute and is represented with period[freq] like period[D] or period[M], using frequency strings.

In [319]:
pi = pd.period_range("2016-01-01", periods=3, freq="M")
pi

PeriodIndex(['2016-01', '2016-02', '2016-03'], dtype='period[M]')

In [320]:
pi.dtype

period[M]

- The period dtype can be used in .astype(...). It allows one to change the freq of a PeriodIndex like .asfreq() and convert a DatetimeIndex to PeriodIndex like to_period():

In [321]:
# change monthly freq to daily freq
pi.astype("period[D]")

PeriodIndex(['2016-01-31', '2016-02-29', '2016-03-31'], dtype='period[D]')

In [322]:
# convert to DatetimeIndex
pi.astype("datetime64[ns]")

DatetimeIndex(['2016-01-01', '2016-02-01', '2016-03-01'], dtype='datetime64[ns]', freq='MS')

In [324]:
# convert to PeriodIndex
dti = pd.date_range("2011-01-01", freq="ME", periods=3)
dti

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31'], dtype='datetime64[ns]', freq='ME')

In [325]:
dti.astype("period[M]")

PeriodIndex(['2011-01', '2011-02', '2011-03'], dtype='period[M]')

#### PeriodIndex partial string indexing
- PeriodIndex now supports partial string slicing with non-monotonic indexes.

- You can pass in dates and strings to Series and DataFrame with PeriodIndex, in the same manner as DatetimeIndex. For details, refer to DatetimeIndex Partial String Indexing.

In [326]:
ps["2011-01"]

np.float64(0.8021782484192926)

In [327]:
ps[datetime.datetime(2011, 12, 25):]

2011-12   -0.911262
2012-01    0.784511
Freq: M, dtype: float64

In [328]:
ps["10/31/2011":"12/31/2011"]

2011-10    0.477741
2011-11    1.711656
2011-12   -0.911262
Freq: M, dtype: float64

- Passing a string representing a lower frequency than PeriodIndex returns partial sliced data.

In [329]:
ps["2011"]

2011-01    0.802178
2011-02    1.037438
2011-03    0.032244
2011-04    0.136508
2011-05   -0.833892
2011-06    1.204212
2011-07   -1.083029
2011-08   -0.156050
2011-09   -0.447367
2011-10    0.477741
2011-11    1.711656
2011-12   -0.911262
Freq: M, dtype: float64

In [330]:
dfp = pd.DataFrame(
    np.random.randn(600, 1),
    columns=["A"],
    index=pd.period_range("2013-01-01 9:00", periods=600, freq="min"),
)
dfp

,A
2013-01-01 09:00,0.045779
2013-01-01 09:01,0.267938
2013-01-01 09:02,-1.158871
2013-01-01 09:03,2.057727
2013-01-01 09:04,-0.595679
...,...
2013-01-01 18:55,0.226862
2013-01-01 18:56,-1.656342
2013-01-01 18:57,-0.546780
2013-01-01 18:58,-1.773293


In [331]:
dfp.loc["2013-01-01 10h"]

,A
2013-01-01 10:00,0.019183
2013-01-01 10:01,2.679683
2013-01-01 10:02,-0.723785
2013-01-01 10:03,0.326775
2013-01-01 10:04,-1.206009
2013-01-01 10:05,0.369217
2013-01-01 10:06,1.908229
2013-01-01 10:07,1.239513
2013-01-01 10:08,1.332314
2013-01-01 10:09,-0.548558


- As with DatetimeIndex, the endpoints will be included in the result. The example below slices data starting from 10:00 to 11:59.

In [332]:
dfp["2013-01-01 10h":"2013-01-01 11h"]

,A
2013-01-01 10:00,0.019183
2013-01-01 10:01,2.679683
2013-01-01 10:02,-0.723785
2013-01-01 10:03,0.326775
2013-01-01 10:04,-1.206009
...,...
2013-01-01 11:55,-0.326448
2013-01-01 11:56,1.128193
2013-01-01 11:57,-0.772802
2013-01-01 11:58,-1.530819


#### Frequency conversion and resampling with PeriodIndex
- The frequency of Period and PeriodIndex can be converted via the asfreq method. Let’s start with the fiscal year 2011, ending in December:

In [333]:
p = pd.Period("2011", freq="Y-DEC")
p 

Period('2011', 'Y-DEC')

- We can convert it to a monthly frequency. Using the how parameter, we can specify whether to return the starting or ending month:

In [334]:
p.asfreq("M", how="start")

Period('2011-01', 'M')

In [335]:
p.asfreq("M", how="end")

Period('2011-12', 'M')

- The shorthands ‘s’ and ‘e’ are provided for convenience:

In [336]:
p.asfreq("M", "s")

Period('2011-01', 'M')

In [337]:
p.asfreq("M", "e")

Period('2011-12', 'M')

- Converting to a “super-period” (e.g., annual frequency is a super-period of quarterly frequency) automatically returns the super-period that includes the input period:

In [338]:
p = pd.Period("2011-12", freq="M")
p.asfreq("Y-NOV")

Period('2012', 'Y-NOV')

- **Note** that since we converted to an annual frequency that ends the year in November, the monthly period of December 2011 is actually in the 2012 Y-NOV period.

- Period conversions with anchored frequencies are particularly useful for working with various quarterly data common to economics, business, and other fields. Many organizations define quarters relative to the month in which their fiscal year starts and ends. Thus, first quarter of 2011 could start in 2010 or a few months into 2011. Via anchored frequencies, pandas works for all quarterly frequencies Q-JAN through Q-DEC.

- Q-DEC define regular calendar quarters:

In [339]:
p = pd.Period("2012Q1", freq="Q-DEC")
p.asfreq("D", "s")


Period('2012-01-01', 'D')

In [340]:
p.asfreq("D", "e")

Period('2012-03-31', 'D')

- Q-MAR defines fiscal year end in March:

In [341]:
p = pd.Period("2011Q4", freq="Q-MAR")
p.asfreq("D", "s")

Period('2011-01-01', 'D')

In [342]:
p.asfreq("D", "e")

Period('2011-03-31', 'D')

#### Converting between representations
- Timestamped data can be converted to PeriodIndex-ed data using to_period and vice-versa using to_timestamp:

In [343]:
rng = pd.date_range("1/1/2012", periods=5, freq="ME")
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-01-31   -0.032520
2012-02-29    0.448457
2012-03-31   -1.392803
2012-04-30    0.679323
2012-05-31   -0.405056
Freq: ME, dtype: float64

In [344]:
ps = ts.to_period()
ps

2012-01   -0.032520
2012-02    0.448457
2012-03   -1.392803
2012-04    0.679323
2012-05   -0.405056
Freq: M, dtype: float64

In [345]:
ps.to_timestamp()

2012-01-01   -0.032520
2012-02-01    0.448457
2012-03-01   -1.392803
2012-04-01    0.679323
2012-05-01   -0.405056
Freq: MS, dtype: float64

- Remember that ‘s’ and ‘e’ can be used to return the timestamps at the start or end of the period:

In [346]:
ps.to_timestamp("D", how="s")

2012-01-01   -0.032520
2012-02-01    0.448457
2012-03-01   -1.392803
2012-04-01    0.679323
2012-05-01   -0.405056
Freq: MS, dtype: float64

- Converting between period and timestamp enables some convenient arithmetic functions to be used. In the following example, we convert a quarterly frequency with year ending in November to 9am of the end of the month following the quarter end:

In [347]:
prng = pd.period_range("1990Q1", "2000Q4", freq="Q-NOV")
ts = pd.Series(np.random.randn(len(prng)), prng)
ts.index = (prng.asfreq("M", "e") + 1).asfreq("h", "s") + 9
ts.head()


1990-03-01 09:00   -1.916962
1990-06-01 09:00    0.466212
1990-09-01 09:00   -0.628176
1990-12-01 09:00   -0.637134
1991-03-01 09:00    0.681354
Freq: h, dtype: float64

#### Representing out-of-bounds spans
- If you have data that is outside of the Timestamp bounds, see Timestamp limitations, then you can use a PeriodIndex and/or Series of Periods to do computations.

In [348]:
span = pd.period_range("1215-01-01", "1381-01-01", freq="D")
span

PeriodIndex(['1215-01-01', '1215-01-02', '1215-01-03', '1215-01-04',
             '1215-01-05', '1215-01-06', '1215-01-07', '1215-01-08',
             '1215-01-09', '1215-01-10',
             ...
             '1380-12-23', '1380-12-24', '1380-12-25', '1380-12-26',
             '1380-12-27', '1380-12-28', '1380-12-29', '1380-12-30',
             '1380-12-31', '1381-01-01'],
            dtype='period[D]', length=60632)

- To convert from an int64 based YYYYMMDD representation.

In [349]:
s = pd.Series([20121231, 20141130, 99991231])
s

0    20121231
1    20141130
2    99991231
dtype: int64

In [350]:
def conv(x):
    return pd.Period(year=x // 10000, month=x // 100 % 100, day=x % 100, freq="D")

s.apply(conv)

0    2012-12-31
1    2014-11-30
2    9999-12-31
dtype: period[D]

In [351]:
s.apply(conv)[2]

Period('9999-12-31', 'D')

- These can easily be converted to a PeriodIndex:

In [352]:
span = pd.PeriodIndex(s.apply(conv))
span

PeriodIndex(['2012-12-31', '2014-11-30', '9999-12-31'], dtype='period[D]')

#### Time zone handling
- pandas provides rich support for working with timestamps in different time zones using the pytz and dateutil libraries or datetime.timezone objects from the standard library.

#### Working with time zones
- By default, pandas objects are time zone unaware:

In [353]:
rng = pd.date_range("3/6/2012 00:00", periods=15, freq="D")
rng.tz is None

True

- To localize these dates to a time zone (assign a particular time zone to a naive date), you can use the tz_localize method or the tz keyword argument in date_range(), Timestamp, or DatetimeIndex. You can either pass pytz or dateutil time zone objects or Olson time zone database strings. Olson time zone strings will return pytz time zone objects by default. To return dateutil time zone objects, append dateutil/ before the string.
    - In pytz you can find a list of common (and less common) time zones using from pytz import common_timezones, all_timezones.
    - dateutil uses the OS time zones so there isn’t a fixed list available. For common zones, the names are the same as pytz.

In [354]:
import dateutil

rng_pytz = pd.date_range("3/6/2012 00:00", periods=3, freq="D", tz="Europe/London")

rng_pytz.tz

<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>

In [355]:
rng_dateutil = pd.date_range("3/6/2012 00:00", periods=3, freq="D")
rng_dateutil = rng_dateutil.tz_localize("dateutil/Europe/London")
rng_dateutil.tz

tzfile('GB-Eire')

In [357]:
# dateutil - utc special case
rng_utc = pd.date_range(
    "3/6/2012 00:00",
    periods=3,
    freq="D",
    tz=dateutil.tz.tzutc(),
)
rng_utc.tz

tzutc()

In [358]:
# datetime.timezone
rng_utc = pd.date_range(
    "3/6/2012 00:00",
    periods=3,
    freq="D",
    tz=datetime.timezone.utc,
)
rng_utc.tz

datetime.timezone.utc

- Note that the UTC time zone is a special case in dateutil and should be constructed explicitly as an instance of dateutil.tz.tzutc. You can also construct other time zones objects explicitly first.

In [359]:
import pytz

tz_pytz = pytz.timezone("Europe/London")
rng_pytz = pd.date_range("3/6/2012 00:00", periods=3, freq="D")
rng_pytz = rng_pytz.tz_localize(tz_pytz)
rng_pytz.tz == tz_pytz


True

In [360]:
tz_dateutil = dateutil.tz.gettz("Europe/London")
rng_dateutil = pd.date_range("3/6/2012 00:00", periods=3, freq="D", tz=tz_dateutil)
rng_dateutil.tz == tz_dateutil



True

- To convert a time zone aware pandas object from one time zone to another, you can use the tz_convert method.

In [361]:
rng_pytz.tz_convert("US/Eastern")

DatetimeIndex(['2012-03-05 19:00:00-05:00', '2012-03-06 19:00:00-05:00',
               '2012-03-07 19:00:00-05:00'],
              dtype='datetime64[ns, US/Eastern]', freq=None)

- **Note**:  When using pytz time zones, DatetimeIndex will construct a different time zone object than a Timestamp for the same time zone input. A DatetimeIndex can hold a collection of Timestamp objects that may have different UTC offsets and cannot be succinctly represented by one pytz time zone instance while one Timestamp represents one point in time with a specific UTC offset.

In [362]:
dti = pd.date_range("2019-01-01", periods=3, freq="D", tz="US/Pacific")
dti.tz

<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>

In [363]:
ts = pd.Timestamp("2019-01-01", tz="US/Pacific")
ts.tz

<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>

- **Warning**: Be wary of conversions between libraries. For some time zones, pytz and dateutil have different definitions of the zone. This is more of a problem for unusual time zones than for ‘standard’ zones like US/Eastern.
- **Warning**: Be aware that a time zone definition across versions of time zone libraries may not be considered equal. This may cause problems when working with stored data that is localized using one version and operated on with a different version. See here for how to handle such a situation.
- **Warning**: For pytz time zones, it is incorrect to pass a time zone object directly into the datetime.datetime constructor (e.g., datetime.datetime(2011, 1, 1), tzinfo=pytz.timezone('US/Eastern')). Instead, the datetime needs to be localized using the localize method on the pytz time zone object.
- **Warning**: Be aware that for times in the future, correct conversion between time zones (and UTC) cannot be guaranteed by any time zone library because a timezone’s offset from UTC may be changed by the respective government.
- **Warning**: If you are using dates beyond 2038-01-18, due to current deficiencies in the underlying libraries caused by the year 2038 problem, daylight saving time (DST) adjustments to timezone aware dates will not be applied. If and when the underlying libraries are fixed, the DST transitions will be applied.

- For example, for two dates that are in British Summer Time (and so would normally be GMT+1), both the following asserts evaluate as true:

In [364]:
d_2037 = "2037-03-31T010101"
d_2038 = "2038-03-31T010101"
DST = "Europe/London"
assert pd.Timestamp(d_2037, tz=DST) != pd.Timestamp(d_2037, tz="GMT")
assert pd.Timestamp(d_2038, tz=DST) == pd.Timestamp(d_2038, tz="GMT")


- Under the hood, all timestamps are stored in UTC. Values from a time zone aware DatetimeIndex or Timestamp will have their fields (day, hour, minute, etc.) localized to the time zone. However, timestamps with the same UTC value are still considered to be equal even if they are in different time zones:

In [365]:
rng_eastern = rng_utc.tz_convert("US/Eastern")
rng_berlin = rng_utc.tz_convert("Europe/Berlin")
rng_eastern[2]


Timestamp('2012-03-07 19:00:00-0500', tz='US/Eastern')

In [366]:
rng_berlin[2]


Timestamp('2012-03-08 01:00:00+0100', tz='Europe/Berlin')

In [367]:
rng_eastern[2] == rng_berlin[2]

True

- Operations between Series in different time zones will yield UTC Series, aligning the data on the UTC timestamps:

In [368]:
ts_utc = pd.Series(range(3), pd.date_range("20130101", periods=3, tz="UTC"))
eastern = ts_utc.tz_convert("US/Eastern")
berlin = ts_utc.tz_convert("Europe/Berlin")
result = eastern + berlin
result

2013-01-01 00:00:00+00:00    0
2013-01-02 00:00:00+00:00    2
2013-01-03 00:00:00+00:00    4
Freq: D, dtype: int64

In [369]:
result.index

DatetimeIndex(['2013-01-01 00:00:00+00:00', '2013-01-02 00:00:00+00:00',
               '2013-01-03 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

- To remove time zone information, use tz_localize(None) or tz_convert(None). tz_localize(None) will remove the time zone yielding the local time representation. tz_convert(None) will remove the time zone after converting to UTC time.

In [370]:
didx = pd.date_range(start="2014-08-01 09:00", freq="h", periods=3, tz="US/Eastern")
didx

DatetimeIndex(['2014-08-01 09:00:00-04:00', '2014-08-01 10:00:00-04:00',
               '2014-08-01 11:00:00-04:00'],
              dtype='datetime64[ns, US/Eastern]', freq='h')

In [371]:
didx.tz_localize(None)

DatetimeIndex(['2014-08-01 09:00:00', '2014-08-01 10:00:00',
               '2014-08-01 11:00:00'],
              dtype='datetime64[ns]', freq=None)

In [372]:
didx.tz_convert(None)

DatetimeIndex(['2014-08-01 13:00:00', '2014-08-01 14:00:00',
               '2014-08-01 15:00:00'],
              dtype='datetime64[ns]', freq='h')

In [373]:
# tz_convert(None) is identical to tz_convert('UTC').tz_localize(None)
didx.tz_convert("UTC").tz_localize(None)

DatetimeIndex(['2014-08-01 13:00:00', '2014-08-01 14:00:00',
               '2014-08-01 15:00:00'],
              dtype='datetime64[ns]', freq=None)

#### Fold
- For ambiguous times, pandas supports explicitly specifying the keyword-only fold argument. Due to daylight saving time, one wall clock time can occur twice when shifting from summer to winter time; fold describes whether the datetime-like corresponds to the first (0) or the second time (1) the wall clock hits the ambiguous time. Fold is supported only for constructing from naive datetime.datetime (see datetime documentation for details) or from Timestamp or for constructing from components (see below). Only dateutil timezones are supported (see dateutil documentation for dateutil methods that deal with ambiguous datetimes) as pytz timezones do not support fold (see pytz documentation for details on how pytz deals with ambiguous datetimes). To localize an ambiguous datetime with pytz, please use Timestamp.tz_localize(). In general, we recommend to rely on Timestamp.tz_localize() when localizing ambiguous datetimes if you need direct control over how they are handled.

In [374]:
pd.Timestamp(
    datetime.datetime(2019, 10, 27, 1, 30, 0, 0),
    tz="dateutil/Europe/London",
    fold=0,
)

Timestamp('2019-10-27 01:30:00+0100', tz='dateutil/GB-Eire')

In [375]:
pd.Timestamp(
    year=2019,
    month=10,
    day=27,
    hour=1,
    minute=30,
    tz="dateutil/Europe/London",
    fold=1,
)

Timestamp('2019-10-27 01:30:00+0000', tz='dateutil/GB-Eire')

#### Ambiguous times when localizing
- tz_localize may not be able to determine the UTC offset of a timestamp because daylight savings time (DST) in a local time zone causes some times to occur twice within one day (“clocks fall back”). The following options are available:
    - 'raise': Raises a pytz.AmbiguousTimeError (the default behavior)
    - 'infer': Attempt to determine the correct offset base on the monotonicity of the timestamps
    - 'NaT': Replaces ambiguous times with NaT
    - bool: True represents a DST time, False represents non-DST time. An array-like of bool values is supported for a sequence of times.

In [376]:
rng_hourly = pd.DatetimeIndex(
    ["11/06/2011 00:00", "11/06/2011 01:00", "11/06/2011 01:00", "11/06/2011 02:00"]
)

- This will fail as there are ambiguous times ('11/06/2011 01:00')

In [378]:
try:
    rng_hourly.tz_localize('US/Eastern')
except Exception as e:
    print(e)

Cannot infer dst time from 2011-11-06 01:00:00, try using the 'ambiguous' argument


- Handle these ambiguous times by specifying the following.

In [379]:
rng_hourly.tz_localize("US/Eastern", ambiguous="infer")

DatetimeIndex(['2011-11-06 00:00:00-04:00', '2011-11-06 01:00:00-04:00',
               '2011-11-06 01:00:00-05:00', '2011-11-06 02:00:00-05:00'],
              dtype='datetime64[ns, US/Eastern]', freq=None)

In [380]:
rng_hourly.tz_localize("US/Eastern", ambiguous="NaT")

DatetimeIndex(['2011-11-06 00:00:00-04:00', 'NaT', 'NaT',
               '2011-11-06 02:00:00-05:00'],
              dtype='datetime64[ns, US/Eastern]', freq=None)

In [381]:
rng_hourly.tz_localize("US/Eastern", ambiguous=[True, True, False, False])


DatetimeIndex(['2011-11-06 00:00:00-04:00', '2011-11-06 01:00:00-04:00',
               '2011-11-06 01:00:00-05:00', '2011-11-06 02:00:00-05:00'],
              dtype='datetime64[ns, US/Eastern]', freq=None)

#### Nonexistent times when localizing
- A DST transition may also shift the local time ahead by 1 hour creating nonexistent local times (“clocks spring forward”). The behavior of localizing a timeseries with nonexistent times can be controlled by the nonexistent argument. The following options are available:
    - 'raise': Raises a pytz.NonExistentTimeError (the default behavior)
    - 'NaT': Replaces nonexistent times with NaT
    - 'shift_forward': Shifts nonexistent times forward to the closest real time
    - 'shift_backward': Shifts nonexistent times backward to the closest real time
    - timedelta object: Shifts nonexistent times by the timedelta duration

In [382]:
dti = pd.date_range(start="2015-03-29 02:30:00", periods=3, freq="h")


- Localization of nonexistent times will raise an error by default.

In [383]:
try:
    dti.tz_localize('Europe/Warsaw')
except Exception as e:
    print(e)

2015-03-29 02:30:00


- Transform nonexistent times to NaT or shift the times.

In [384]:
dti

DatetimeIndex(['2015-03-29 02:30:00', '2015-03-29 03:30:00',
               '2015-03-29 04:30:00'],
              dtype='datetime64[ns]', freq='h')

In [385]:
dti.tz_localize("Europe/Warsaw", nonexistent="shift_forward")


DatetimeIndex(['2015-03-29 03:00:00+02:00', '2015-03-29 03:30:00+02:00',
               '2015-03-29 04:30:00+02:00'],
              dtype='datetime64[ns, Europe/Warsaw]', freq=None)

In [386]:
dti.tz_localize("Europe/Warsaw", nonexistent="shift_backward")


DatetimeIndex(['2015-03-29 01:59:59.999999999+01:00',
                         '2015-03-29 03:30:00+02:00',
                         '2015-03-29 04:30:00+02:00'],
              dtype='datetime64[ns, Europe/Warsaw]', freq=None)

In [387]:
dti.tz_localize("Europe/Warsaw", nonexistent=pd.Timedelta(1, unit="h"))


DatetimeIndex(['2015-03-29 03:30:00+02:00', '2015-03-29 03:30:00+02:00',
               '2015-03-29 04:30:00+02:00'],
              dtype='datetime64[ns, Europe/Warsaw]', freq=None)

In [388]:
dti.tz_localize("Europe/Warsaw", nonexistent="NaT")


DatetimeIndex(['NaT', '2015-03-29 03:30:00+02:00',
               '2015-03-29 04:30:00+02:00'],
              dtype='datetime64[ns, Europe/Warsaw]', freq=None)

#### Time zone Series operations
- A Series with time zone naive values is represented with a dtype of datetime64[ns].

In [389]:
s_naive = pd.Series(pd.date_range("20130101", periods=3))
s_naive

0   2013-01-01
1   2013-01-02
2   2013-01-03
dtype: datetime64[ns]

- A Series with a time zone aware values is represented with a dtype of datetime64[ns, tz] where tz is the time zone

In [390]:
s_aware = pd.Series(pd.date_range("20130101", periods=3, tz="US/Eastern"))
s_aware

0   2013-01-01 00:00:00-05:00
1   2013-01-02 00:00:00-05:00
2   2013-01-03 00:00:00-05:00
dtype: datetime64[ns, US/Eastern]

- Both of these Series time zone information can be manipulated via the .dt accessor, see the dt accessor section.

- For example, to localize and convert a naive stamp to time zone aware.

In [391]:
s_naive.dt.tz_localize("UTC").dt.tz_convert("US/Eastern")


0   2012-12-31 19:00:00-05:00
1   2013-01-01 19:00:00-05:00
2   2013-01-02 19:00:00-05:00
dtype: datetime64[ns, US/Eastern]

- Time zone information can also be manipulated using the astype method. This method can convert between different timezone-aware dtypes.

In [392]:
# convert to a new time zone
s_aware.astype("datetime64[ns, CET]")

0   2013-01-01 06:00:00+01:00
1   2013-01-02 06:00:00+01:00
2   2013-01-03 06:00:00+01:00
dtype: datetime64[ns, CET]

- **Note**: Using Series.to_numpy() on a Series, returns a NumPy array of the data. NumPy does not currently support time zones (even though it is printing in the local time zone!), therefore an object array of Timestamps is returned for time zone aware data:

In [393]:
s_naive.to_numpy()

array(['2013-01-01T00:00:00.000000000', '2013-01-02T00:00:00.000000000',
       '2013-01-03T00:00:00.000000000'], dtype='datetime64[ns]')

In [394]:
s_aware.to_numpy()

array([Timestamp('2013-01-01 00:00:00-0500', tz='US/Eastern'),
       Timestamp('2013-01-02 00:00:00-0500', tz='US/Eastern'),
       Timestamp('2013-01-03 00:00:00-0500', tz='US/Eastern')],
      dtype=object)

- By converting to an object array of Timestamps, it preserves the time zone information. For example, when converting back to a Series:

In [395]:
pd.Series(s_aware.to_numpy())

0   2013-01-01 00:00:00-05:00
1   2013-01-02 00:00:00-05:00
2   2013-01-03 00:00:00-05:00
dtype: datetime64[ns, US/Eastern]

- However, if you want an actual NumPy datetime64[ns] array (with the values converted to UTC) instead of an array of objects, you can specify the dtype argument:

In [396]:
s_aware.to_numpy(dtype="datetime64[ns]")

array(['2013-01-01T05:00:00.000000000', '2013-01-02T05:00:00.000000000',
       '2013-01-03T05:00:00.000000000'], dtype='datetime64[ns]')

#### Time deltas
- From https://pandas.pydata.org/docs/user_guide/timedeltas.html
- Timedeltas are differences in times, expressed in difference units, e.g. days, hours, minutes, seconds. They can be both positive and negative.

- Timedelta is a subclass of datetime.timedelta, and behaves in a similar manner, but allows compatibility with np.timedelta64 types as well as a host of custom representation, parsing, and attributes.

#### Parsing
- You can construct a Timedelta scalar through various arguments, including ISO 8601 Duration strings.

In [397]:
import datetime

# strings
pd.Timedelta("1 days")

Timedelta('1 days 00:00:00')

In [398]:
pd.Timedelta("1 days 00:00:00")

Timedelta('1 days 00:00:00')

In [399]:
pd.Timedelta("1 days 2 hours")

Timedelta('1 days 02:00:00')

In [400]:
pd.Timedelta("-1 days 2 min 3us")


Timedelta('-2 days +23:57:59.999997')

In [401]:
# like datetime.timedelta
# note: these MUST be specified as keyword arguments
pd.Timedelta(days=1, seconds=1)

Timedelta('1 days 00:00:01')

In [402]:
# integers with a unit
pd.Timedelta(1, unit="d")

Timedelta('1 days 00:00:00')

In [403]:
# from a datetime.timedelta/np.timedelta64
pd.Timedelta(datetime.timedelta(days=1, seconds=1))

Timedelta('1 days 00:00:01')

In [404]:
pd.Timedelta(np.timedelta64(1, "ms"))

Timedelta('0 days 00:00:00.001000')

In [405]:
# negative Timedeltas have this string repr
# to be more consistent with datetime.timedelta conventions
pd.Timedelta("-1us")

Timedelta('-1 days +23:59:59.999999')

In [406]:
# a NaT
pd.Timedelta("nan")

NaT

In [407]:
pd.Timedelta("nat")

NaT

In [408]:
# ISO 8601 Duration strings
pd.Timedelta("P0DT0H1M0S")

Timedelta('0 days 00:01:00')

In [409]:
pd.Timedelta("P0DT0H0M0.000000123S")

Timedelta('0 days 00:00:00.000000123')

- DateOffsets (Day, Hour, Minute, Second, Milli, Micro, Nano) can also be used in construction.

In [410]:
pd.Timedelta(pd.offsets.Second(2))

Timedelta('0 days 00:00:02')

- Further, operations among the scalars yield another scalar Timedelta.

In [411]:
pd.Timedelta(pd.offsets.Day(2)) + pd.Timedelta(pd.offsets.Second(2)) + pd.Timedelta(
    "00:00:00.000123"
)

Timedelta('2 days 00:00:02.000123')

#### to_timedelta
- Using the top-level pd.to_timedelta, you can convert a scalar, array, list, or Series from a recognized timedelta format / value into a Timedelta type. It will construct Series if the input is a Series, a scalar if the input is scalar-like, otherwise it will output a TimedeltaIndex.

- You can parse a single string to a Timedelta:

In [412]:
pd.to_timedelta("1 days 06:05:01.00003")

Timedelta('1 days 06:05:01.000030')

In [413]:
pd.to_timedelta("15.5us")

Timedelta('0 days 00:00:00.000015500')

- or a list/array of strings:

In [414]:
pd.to_timedelta(["1 days 06:05:01.00003", "15.5us", "nan"])

TimedeltaIndex(['1 days 06:05:01.000030', '0 days 00:00:00.000015500', NaT], dtype='timedelta64[ns]', freq=None)

- The unit keyword argument specifies the unit of the Timedelta if the input is numeric:

In [415]:
pd.to_timedelta(np.arange(5), unit="s")

TimedeltaIndex(['0 days 00:00:00', '0 days 00:00:01', '0 days 00:00:02',
                '0 days 00:00:03', '0 days 00:00:04'],
               dtype='timedelta64[ns]', freq=None)

In [416]:
pd.to_timedelta(np.arange(5), unit="d")

TimedeltaIndex(['0 days', '1 days', '2 days', '3 days', '4 days'], dtype='timedelta64[ns]', freq=None)

- **Warning**: If a string or array of strings is passed as an input then the unit keyword argument will be ignored. If a string without units is passed then the default unit of nanoseconds is assumed.

#### Timedelta limitations
- pandas represents Timedeltas in nanosecond resolution using 64 bit integers. As such, the 64 bit integer limits determine the Timedelta limits.

In [417]:
pd.Timedelta.min

Timedelta('-106752 days +00:12:43.145224193')

In [418]:
pd.Timedelta.max

Timedelta('106751 days 23:47:16.854775807')

#### Operations
- You can operate on Series/DataFrames and construct timedelta64[ns] Series through subtraction operations on datetime64[ns] Series, or Timestamps.

In [419]:
s = pd.Series(pd.date_range("2012-1-1", periods=3, freq="D"))
td = pd.Series([pd.Timedelta(days=i) for i in range(3)])
df = pd.DataFrame({"A": s, "B": td})
df

,A,B
0,2012-01-01,0 days
1,2012-01-02,1 days
2,2012-01-03,2 days


In [420]:
df["C"] = df["A"] + df["B"]
df

,A,B,C
0,2012-01-01,0 days,2012-01-01
1,2012-01-02,1 days,2012-01-03
2,2012-01-03,2 days,2012-01-05


In [421]:
df.dtypes

A     datetime64[ns]
B    timedelta64[ns]
C     datetime64[ns]
dtype: object

In [422]:
s - s.max()

0   -2 days
1   -1 days
2    0 days
dtype: timedelta64[ns]

In [423]:
s - datetime.datetime(2011, 1, 1, 3, 5)

0   364 days 20:55:00
1   365 days 20:55:00
2   366 days 20:55:00
dtype: timedelta64[ns]

In [424]:
s + datetime.timedelta(minutes=5)

0   2012-01-01 00:05:00
1   2012-01-02 00:05:00
2   2012-01-03 00:05:00
dtype: datetime64[ns]

In [425]:
s + pd.offsets.Minute(5)

0   2012-01-01 00:05:00
1   2012-01-02 00:05:00
2   2012-01-03 00:05:00
dtype: datetime64[ns]

In [426]:
s + pd.offsets.Minute(5) + pd.offsets.Milli(5)

0   2012-01-01 00:05:00.005
1   2012-01-02 00:05:00.005
2   2012-01-03 00:05:00.005
dtype: datetime64[ns]

- Operations with scalars from a timedelta64[ns] series:

In [427]:
y = s - s[0]
y

0   0 days
1   1 days
2   2 days
dtype: timedelta64[ns]

- Series of timedeltas with NaT values are supported:

In [428]:
y = s - s.shift()
y

0      NaT
1   1 days
2   1 days
dtype: timedelta64[ns]

- Elements can be set to NaT using np.nan analogously to datetimes:

In [429]:
y[1] = np.nan
y

0      NaT
1      NaT
2   1 days
dtype: timedelta64[ns]

- Operands can also appear in a reversed order (a singular object operated with a Series):

In [430]:
s.max() - s

0   2 days
1   1 days
2   0 days
dtype: timedelta64[ns]

In [431]:
datetime.datetime(2011, 1, 1, 3, 5) - s

0   -365 days +03:05:00
1   -366 days +03:05:00
2   -367 days +03:05:00
dtype: timedelta64[ns]

In [432]:
datetime.timedelta(minutes=5) + s

0   2012-01-01 00:05:00
1   2012-01-02 00:05:00
2   2012-01-03 00:05:00
dtype: datetime64[ns]

- min, max and the corresponding idxmin, idxmax operations are supported on frames:

In [433]:
A = s - pd.Timestamp("20120101") - pd.Timedelta("00:05:05")
B = s - pd.Series(pd.date_range("2012-1-2", periods=3, freq="D"))
df = pd.DataFrame({"A": A, "B": B})
df

,A,B
0,-1 days +23:54:55,-1 days
1,0 days 23:54:55,-1 days
2,1 days 23:54:55,-1 days


In [434]:
df.min()

A   -1 days +23:54:55
B   -1 days +00:00:00
dtype: timedelta64[ns]

In [435]:
df.min(axis=1)

0   -1 days
1   -1 days
2   -1 days
dtype: timedelta64[ns]

In [436]:
df.idxmin()

A    0
B    0
dtype: int64

In [437]:
df.idxmax()

A    2
B    0
dtype: int64

- min, max, idxmin, idxmax operations are supported on Series as well. A scalar result will be a Timedelta.

In [438]:
df.min().max()

Timedelta('-1 days +23:54:55')

In [439]:
df.min(axis=1).min()

Timedelta('-1 days +00:00:00')

In [440]:
df.min().idxmax()

'A'

In [441]:
df.min(axis=1).idxmin()

0

- You can fillna on timedeltas, passing a timedelta to get a particular value.

In [442]:
y.fillna(pd.Timedelta(0))

0   0 days
1   0 days
2   1 days
dtype: timedelta64[ns]

In [443]:
y.fillna(pd.Timedelta(10, unit="s"))

0   0 days 00:00:10
1   0 days 00:00:10
2   1 days 00:00:00
dtype: timedelta64[ns]

In [444]:
y.fillna(pd.Timedelta("-1 days, 00:00:05"))

0   -1 days +00:00:05
1   -1 days +00:00:05
2     1 days 00:00:00
dtype: timedelta64[ns]

- You can also negate, multiply and use abs on Timedeltas:

In [445]:
td1 = pd.Timedelta("-1 days 2 hours 3 seconds")
td1

Timedelta('-2 days +21:59:57')

In [446]:
-1 * td1

Timedelta('1 days 02:00:03')

In [447]:
-td1

Timedelta('1 days 02:00:03')

In [448]:
abs(td1)

Timedelta('1 days 02:00:03')

#### Reductions
- Numeric reduction operation for timedelta64[ns] will return Timedelta objects. As usual NaT are skipped during evaluation.

In [449]:
y2 = pd.Series(
    pd.to_timedelta(["-1 days +00:00:05", "nat", "-1 days +00:00:05", "1 days"])
)
y2

0   -1 days +00:00:05
1                 NaT
2   -1 days +00:00:05
3     1 days 00:00:00
dtype: timedelta64[ns]

In [450]:
y2.mean()

Timedelta('-1 days +16:00:03.333333334')

In [451]:
y2.median()

Timedelta('-1 days +00:00:05')

In [452]:
y2.quantile(0.1)

Timedelta('-1 days +00:00:05')

In [453]:
y2.sum()

Timedelta('-1 days +00:00:10')

#### Frequency conversion
- Timedelta Series and TimedeltaIndex, and Timedelta can be converted to other frequencies by astyping to a specific timedelta dtype.

In [454]:
december = pd.Series(pd.date_range("20121201", periods=4))
january = pd.Series(pd.date_range("20130101", periods=4))
td = january - december
td[2] += datetime.timedelta(minutes=5, seconds=3)
td[3] = np.nan
td

0   31 days 00:00:00
1   31 days 00:00:00
2   31 days 00:05:03
3                NaT
dtype: timedelta64[ns]

In [455]:
td.astype("timedelta64[s]")

0   31 days 00:00:00
1   31 days 00:00:00
2   31 days 00:05:03
3                NaT
dtype: timedelta64[s]

- For timedelta64 resolutions other than the supported “s”, “ms”, “us”, “ns”, an alternative is to divide by another timedelta object. Note that division by the NumPy scalar is true division, while astyping is equivalent of floor division.

In [456]:
# to days
td / np.timedelta64(1, "D")

0    31.000000
1    31.000000
2    31.003507
3          NaN
dtype: float64

- Dividing or multiplying a timedelta64[ns] Series by an integer or integer Series yields another timedelta64[ns] dtypes Series.

In [457]:
td * -1

0   -31 days +00:00:00
1   -31 days +00:00:00
2   -32 days +23:54:57
3                  NaT
dtype: timedelta64[ns]

In [458]:
td * pd.Series([1, 2, 3, 4])

0   31 days 00:00:00
1   62 days 00:00:00
2   93 days 00:15:09
3                NaT
dtype: timedelta64[ns]

- Rounded division (floor-division) of a timedelta64[ns] Series by a scalar Timedelta gives a series of integers.

In [459]:
td // pd.Timedelta(days=3, hours=4)

0    9.0
1    9.0
2    9.0
3    NaN
dtype: float64

In [460]:
pd.Timedelta(days=3, hours=4) // td

0    0.0
1    0.0
2    0.0
3    NaN
dtype: float64

- The mod (%) and divmod operations are defined for Timedelta when operating with another timedelta-like or with a numeric argument.

In [461]:
pd.Timedelta(hours=37) % datetime.timedelta(hours=2)

Timedelta('0 days 01:00:00')

In [462]:
# divmod against a timedelta-like returns a pair (int, Timedelta)
divmod(datetime.timedelta(hours=2), pd.Timedelta(minutes=11))

(10, Timedelta('0 days 00:10:00'))

In [463]:
# divmod against a numeric returns a pair (Timedelta, Timedelta)
divmod(pd.Timedelta(hours=25), 86400000000000)

(Timedelta('0 days 00:00:00.000000001'), Timedelta('0 days 01:00:00'))

#### Attributes
- You can access various components of the Timedelta or TimedeltaIndex directly using the attributes days,seconds,microseconds,nanoseconds. These are identical to the values returned by datetime.timedelta, in that, for example, the .seconds attribute represents the number of seconds >= 0 and < 1 day. These are signed according to whether the Timedelta is signed.

- These operations can also be directly accessed via the .dt property of the Series as well.

- **Note**: Note that the attributes are NOT the displayed values of the Timedelta. Use .components to retrieve the displayed values.

- For a Series:

In [464]:
td.dt.days

0    31.0
1    31.0
2    31.0
3     NaN
dtype: float64

In [465]:
td.dt.seconds

0      0.0
1      0.0
2    303.0
3      NaN
dtype: float64

- You can access the value of the fields for a scalar Timedelta directly.

In [467]:
tds = pd.Timedelta("31 days 5 min 3 sec")
tds.days

31

In [468]:
tds.seconds

303

In [469]:
(-tds).seconds

86097

- You can use the .components property to access a reduced form of the timedelta. This returns a DataFrame indexed similarly to the Series. These are the displayed values of the Timedelta.

In [470]:
td.dt.components

,days,hours,minutes,seconds,milliseconds,microseconds,nanoseconds
0,31.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31.0,0.0,5.0,3.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [471]:
td.dt.components.seconds

0    0.0
1    0.0
2    3.0
3    NaN
Name: seconds, dtype: float64

- You can convert a Timedelta to an ISO 8601 Duration string with the .isoformat method

In [472]:
pd.Timedelta(
    days=6, minutes=50, seconds=3, milliseconds=10, microseconds=10, nanoseconds=12
).isoformat()

'P6DT0H50M3.010010012S'

#### TimedeltaIndex
- To generate an index with time delta, you can use either the TimedeltaIndex or the timedelta_range() constructor.

- Using TimedeltaIndex you can pass string-like, Timedelta, timedelta, or np.timedelta64 objects. Passing np.nan/pd.NaT/nat will represent missing values.

In [473]:
pd.TimedeltaIndex(
    [
        "1 days",
        "1 days, 00:00:05",
        np.timedelta64(2, "D"),
        datetime.timedelta(days=2, seconds=2),
    ]
)

TimedeltaIndex(['1 days 00:00:00', '1 days 00:00:05', '2 days 00:00:00',
                '2 days 00:00:02'],
               dtype='timedelta64[ns]', freq=None)

- The string ‘infer’ can be passed in order to set the frequency of the index as the inferred frequency upon creation:

In [474]:
pd.TimedeltaIndex(["0 days", "10 days", "20 days"], freq="infer")


TimedeltaIndex(['0 days', '10 days', '20 days'], dtype='timedelta64[ns]', freq='10D')

#### Generating ranges of time deltas
- Similar to date_range(), you can construct regular ranges of a TimedeltaIndex using timedelta_range(). The default frequency for timedelta_range is calendar day:

In [475]:
pd.timedelta_range(start="1 days", periods=5)

TimedeltaIndex(['1 days', '2 days', '3 days', '4 days', '5 days'], dtype='timedelta64[ns]', freq='D')

- Various combinations of start, end, and periods can be used with timedelta_range:

In [476]:
pd.timedelta_range(start="1 days", end="5 days")

TimedeltaIndex(['1 days', '2 days', '3 days', '4 days', '5 days'], dtype='timedelta64[ns]', freq='D')

In [477]:
pd.timedelta_range(end="10 days", periods=4)

TimedeltaIndex(['7 days', '8 days', '9 days', '10 days'], dtype='timedelta64[ns]', freq='D')

- The freq parameter can passed a variety of frequency aliases:

In [478]:
pd.timedelta_range(start="1 days", end="2 days", freq="30min")

TimedeltaIndex(['1 days 00:00:00', '1 days 00:30:00', '1 days 01:00:00',
                '1 days 01:30:00', '1 days 02:00:00', '1 days 02:30:00',
                '1 days 03:00:00', '1 days 03:30:00', '1 days 04:00:00',
                '1 days 04:30:00', '1 days 05:00:00', '1 days 05:30:00',
                '1 days 06:00:00', '1 days 06:30:00', '1 days 07:00:00',
                '1 days 07:30:00', '1 days 08:00:00', '1 days 08:30:00',
                '1 days 09:00:00', '1 days 09:30:00', '1 days 10:00:00',
                '1 days 10:30:00', '1 days 11:00:00', '1 days 11:30:00',
                '1 days 12:00:00', '1 days 12:30:00', '1 days 13:00:00',
                '1 days 13:30:00', '1 days 14:00:00', '1 days 14:30:00',
                '1 days 15:00:00', '1 days 15:30:00', '1 days 16:00:00',
                '1 days 16:30:00', '1 days 17:00:00', '1 days 17:30:00',
                '1 days 18:00:00', '1 days 18:30:00', '1 days 19:00:00',
                '1 days 19:30:00', '1 days 20:00:00

In [479]:
pd.timedelta_range(start="1 days", periods=5, freq="2D5h")

TimedeltaIndex(['1 days 00:00:00', '3 days 05:00:00', '5 days 10:00:00',
                '7 days 15:00:00', '9 days 20:00:00'],
               dtype='timedelta64[ns]', freq='53h')

- Specifying start, end, and periods will generate a range of evenly spaced timedeltas from start to end inclusively, with periods number of elements in the resulting TimedeltaIndex:

In [480]:
pd.timedelta_range("0 days", "4 days", periods=5)

TimedeltaIndex(['0 days', '1 days', '2 days', '3 days', '4 days'], dtype='timedelta64[ns]', freq=None)

In [481]:
pd.timedelta_range("0 days", "4 days", periods=10)

TimedeltaIndex(['0 days 00:00:00', '0 days 10:40:00', '0 days 21:20:00',
                '1 days 08:00:00', '1 days 18:40:00', '2 days 05:20:00',
                '2 days 16:00:00', '3 days 02:40:00', '3 days 13:20:00',
                '4 days 00:00:00'],
               dtype='timedelta64[ns]', freq=None)

#### Using the TimedeltaIndex
- Similarly to other of the datetime-like indices, DatetimeIndex and PeriodIndex, you can use TimedeltaIndex as the index of pandas objects.

In [482]:
s = pd.Series(
    np.arange(100),
    index=pd.timedelta_range("1 days", periods=100, freq="h"),
)
s

1 days 00:00:00     0
1 days 01:00:00     1
1 days 02:00:00     2
1 days 03:00:00     3
1 days 04:00:00     4
                   ..
4 days 23:00:00    95
5 days 00:00:00    96
5 days 01:00:00    97
5 days 02:00:00    98
5 days 03:00:00    99
Freq: h, Length: 100, dtype: int64

- Selections work similarly, with coercion on string-likes and slices:

In [483]:
s["1 day":"2 day"]

1 days 00:00:00     0
1 days 01:00:00     1
1 days 02:00:00     2
1 days 03:00:00     3
1 days 04:00:00     4
1 days 05:00:00     5
1 days 06:00:00     6
1 days 07:00:00     7
1 days 08:00:00     8
1 days 09:00:00     9
1 days 10:00:00    10
1 days 11:00:00    11
1 days 12:00:00    12
1 days 13:00:00    13
1 days 14:00:00    14
1 days 15:00:00    15
1 days 16:00:00    16
1 days 17:00:00    17
1 days 18:00:00    18
1 days 19:00:00    19
1 days 20:00:00    20
1 days 21:00:00    21
1 days 22:00:00    22
1 days 23:00:00    23
2 days 00:00:00    24
2 days 01:00:00    25
2 days 02:00:00    26
2 days 03:00:00    27
2 days 04:00:00    28
2 days 05:00:00    29
2 days 06:00:00    30
2 days 07:00:00    31
2 days 08:00:00    32
2 days 09:00:00    33
2 days 10:00:00    34
2 days 11:00:00    35
2 days 12:00:00    36
2 days 13:00:00    37
2 days 14:00:00    38
2 days 15:00:00    39
2 days 16:00:00    40
2 days 17:00:00    41
2 days 18:00:00    42
2 days 19:00:00    43
2 days 20:00:00    44
2 days 21:

In [484]:
s["1 day 01:00:00"]

np.int64(1)

In [485]:
s[pd.Timedelta("1 day 1h")]

np.int64(1)

- Furthermore you can use partial string selection and the range will be inferred:

In [486]:
s["1 day":"1 day 5 hours"]

1 days 00:00:00    0
1 days 01:00:00    1
1 days 02:00:00    2
1 days 03:00:00    3
1 days 04:00:00    4
1 days 05:00:00    5
Freq: h, dtype: int64

#### Operations
- Finally, the combination of TimedeltaIndex with DatetimeIndex allow certain combination operations that are NaT preserving:

In [487]:
tdi = pd.TimedeltaIndex(["1 days", pd.NaT, "2 days"])
tdi.to_list()

[Timedelta('1 days 00:00:00'), NaT, Timedelta('2 days 00:00:00')]

In [488]:
dti = pd.date_range("20130101", periods=3)
dti.to_list()

[Timestamp('2013-01-01 00:00:00'),
 Timestamp('2013-01-02 00:00:00'),
 Timestamp('2013-01-03 00:00:00')]

In [489]:
(dti + tdi).to_list()

[Timestamp('2013-01-02 00:00:00'), NaT, Timestamp('2013-01-05 00:00:00')]

In [490]:
(dti - tdi).to_list()

[Timestamp('2012-12-31 00:00:00'), NaT, Timestamp('2013-01-01 00:00:00')]

#### Conversions
- Similarly to frequency conversion on a Series above, you can convert these indices to yield another Index.

In [491]:
tdi / np.timedelta64(1, "s")

Index([86400.0, nan, 172800.0], dtype='float64')

In [492]:
tdi.astype("timedelta64[s]")

TimedeltaIndex(['1 days', NaT, '2 days'], dtype='timedelta64[s]', freq=None)

- Scalars type ops work as well. These can potentially return a different type of index.

In [493]:
# adding or timedelta and date -> datelike
tdi + pd.Timestamp("20130101")

DatetimeIndex(['2013-01-02', 'NaT', '2013-01-03'], dtype='datetime64[ns]', freq=None)

In [494]:
# subtraction of a date and a timedelta -> datelike
# note that trying to subtract a date from a Timedelta will raise an exception
(pd.Timestamp("20130101") - tdi).to_list()

[Timestamp('2012-12-31 00:00:00'), NaT, Timestamp('2012-12-30 00:00:00')]

In [495]:
# timedelta + timedelta -> timedelta
tdi + pd.Timedelta("10 days")

TimedeltaIndex(['11 days', NaT, '12 days'], dtype='timedelta64[ns]', freq=None)

In [496]:
# division can result in a Timedelta if the divisor is an integer
tdi / 2

TimedeltaIndex(['0 days 12:00:00', NaT, '1 days 00:00:00'], dtype='timedelta64[ns]', freq=None)

In [497]:
# or a float64 Index if the divisor is a Timedelta
tdi / tdi[0]

Index([1.0, nan, 2.0], dtype='float64')

#### Resampling
- Similar to timeseries resampling, we can resample with a TimedeltaIndex.

In [498]:
s.resample("D").mean()

1 days    11.5
2 days    35.5
3 days    59.5
4 days    83.5
5 days    97.5
Freq: D, dtype: float64